In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
import pandas as pd
import ast
from typing import Dict, List, Any, Union
from pandas import DataFrame
import re
import json

In [9]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [10]:
df = pd.read_json('/content/drive/My Drive/final.json', lines=True)
print("DataFrame Shape:", df.shape)
print("\nColumns:", df.columns.tolist())

DataFrame Shape: (51, 18)

Columns: ['business_id', 'name', 'address', 'city', 'state', 'postal_code', 'latitude', 'longitude', 'stars', 'review_count', 'is_open', 'attributes', 'categories', 'hours', 'reviews', 'tips', 'google_reviews', 'menu']


In [11]:
df = df.dropna()

In [12]:
df.head()

business_id                           name                address          city  \
0  4e9tguRcMc4S5hzBlfC62A  The Farm and Fisherman Tavern    1442 Marlton Pike E   Cherry Hill   
1  -pyobEWdpZ6jUp42f3I2xw                   Dolsot House     404 Marlton Pike E   Cherry Hill   
2  onp9X_9RRAEzA-yqdiSOcA             The Jug Handle Inn  1018 S Forklanding Rd   Cinnaminson   
3  RTwDv2oLE3gzAZlf63OPXA               Rat's Restaurant      16 Fairgrounds Rd      Hamilton   
4  UQMUtySn2q9cSObiw5UVsw    The Pop Shop - Collingswood         729 Haddon Ave  Collingswood   

  state  postal_code   latitude  longitude  stars  review_count  is_open  \
0    NJ       8034.0  39.910207 -74.990577    4.0         709.0      1.0   
1    NJ       8034.0  39.912868 -75.002134    4.0         665.0      1.0   
2    NJ       8077.0  39.977343 -75.013310    4.0         604.0      1.0   
3    NJ       8619.0  40.237898 -74.716630    3.5         603.0      1.0   
4    NJ       8108.0  39.916216 -75.067913    3.5         603.0      1.0   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              attributes  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                   {'GoodForKids': 'True', 'RestaurantsGoodForGroups': 'True', 'RestaurantsReservations': 'True', 'BusinessAcceptsCreditCards': 'True', 'RestaurantsAttire': 'u'casual'', 'Alcohol': 'u'full_bar'', 'RestaurantsTakeOut': 'True', 'RestaurantsPriceRange2': '2', 'HasTV': 'True', 'BusinessParking': '{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}', 'NoiseLevel': 'u'average'', 'Ambience': '{'touristy': False, 'hipster': False, 'romantic': False, 'divey': False, 'intimate': False, 'trendy': False, 'upscale': False, 'classy': True, 'casual': True}', 'HappyHour': 'True', 'RestaurantsTableService': 'True', 'WiFi': 'u'free'', 'ByAppointmentOnly': 'False', 'BikeParking': 'True', 'WheelchairAccessible': 'True', 'OutdoorSeating': 'True', 'Caters': 'True', 'BYOB': 'False', 'RestaurantsDelivery': 'None', 'DogsAllowed': 'False', 'Corkage': 'True', 'GoodForMeal': '{'dessert': None, 'latenight': False, 'lunch': True, 'dinner': True, 'brunch': None, 'breakfast': False}'}   
1                                                                                                     

In [24]:
def safe_eval_dict(val: Union[str, Any]) -> Union[Dict, Any]:
    """
    Safely convert a dictionary-like string into a dictionary using ast.literal_eval.

    Args:
        val (Union[str, Any]): A string containing a dictionary representation or any other value

    Returns:
        Union[Dict, Any]: A dictionary if conversion is successful, otherwise returns the original value
    """
    if not isinstance(val, str):
        return val

    val = val.replace("u'", "'").replace('u"', '"').replace("'", '"')

    try:
        return ast.literal_eval(val)
    except:
        return val

def ensure_dicts(d: Union[Dict, Any]) -> Union[Dict, Any]:
    """
    Recursively convert all dictionary-like strings within a dictionary to actual dictionaries.

    Args:
        d (Union[Dict, Any]): A dictionary or any other value to process

    Returns:
        Union[Dict, Any]: A dictionary with all nested dictionary-like strings converted to
        dictionaries, or the original value if input is not a dictionary
    """
    if not isinstance(d, dict):
        return d

    converted = {}
    for k, v in d.items():
        v = safe_eval_dict(v)
        if isinstance(v, dict):
            v = ensure_dicts(v)
        converted[k] = v
    return converted

def analyze_dict_structure(df: DataFrame) -> None:
    """
    Analyze and print the structure of dictionary data stored in a DataFrame's 'attributes' column.
    Converts string representations of dictionaries to actual dictionaries and identifies both
    top-level and nested keys.

    Args:
        df (DataFrame): A pandas DataFrame containing an 'attributes' column with dictionary data

    Returns:
        None: Prints the analysis results to stdout
    """
    df['attributes'] = df['attributes'].apply(safe_eval_dict)
    df['attributes'] = df['attributes'].apply(ensure_dicts)

    outer_keys = set()
    nested_keys = {}

    for attr_dict in df['attributes']:
        if not isinstance(attr_dict, dict):
            continue

        for k, v in attr_dict.items():
            outer_keys.add(k)
            if isinstance(v, dict):
                nested_keys.setdefault(k, set()).update(v.keys())

    print("Top-level keys:")
    print(*sorted(outer_keys), sep='\n')

    print("\nNested keys:")
    for outer_k, inner_set in nested_keys.items():
        for inner_k in sorted(inner_set):
            print(f"{outer_k} - {inner_k}")

In [25]:
df['attributes'] = df['attributes'].apply(safe_eval_dict)
df['attributes'] = df['attributes'].apply(ensure_dicts)
analyze_dict_structure(df)

Top-level keys:
Alcohol
Ambience
BYOB
BYOBCorkage
BestNights
BikeParking
BusinessAcceptsBitcoin
BusinessAcceptsCreditCards
BusinessParking
ByAppointmentOnly
Caters
CoatCheck
Corkage
DogsAllowed
DriveThru
GoodForDancing
GoodForKids
GoodForMeal
HappyHour
HasTV
Music
NoiseLevel
OutdoorSeating
RestaurantsAttire
RestaurantsDelivery
RestaurantsGoodForGroups
RestaurantsPriceRange2
RestaurantsReservations
RestaurantsTableService
RestaurantsTakeOut
Smoking
WheelchairAccessible
WiFi

Nested keys:
BusinessParking - garage
BusinessParking - lot
BusinessParking - street
BusinessParking - valet
BusinessParking - validated
Ambience - casual
Ambience - classy
Ambience - divey
Ambience - hipster
Ambience - intimate
Ambience - romantic
Ambience - touristy
Ambience - trendy
Ambience - upscale
GoodForMeal - breakfast
GoodForMeal - brunch
GoodForMeal - dessert
GoodForMeal - dinner
GoodForMeal - latenight
GoodForMeal - lunch
Music - background_music
Music - dj
Music - jukebox
Music - karaoke
Music - live
Mu

In [26]:
def clean_and_parse_nested(data: Union[str, dict, list, Any]) -> Union[dict, list, Any]:
    """
    Clean and parse nested data structures, converting Python literals to JSON format.

    Args:
        data: Input data to clean and parse

    Returns:
        Cleaned and parsed data structure
    """
    if isinstance(data, str):
        cleaned_data = re.sub(r"u'|'", '"', data)
        cleaned_data = re.sub(r"None", "null", cleaned_data)
        cleaned_data = re.sub(r"False", "false", cleaned_data)
        cleaned_data = re.sub(r"True", "true", cleaned_data)

        try:
            return json.loads(cleaned_data, object_hook=clean_and_parse_nested)
        except json.JSONDecodeError:
            return cleaned_data

    elif isinstance(data, dict):
        return {k: clean_and_parse_nested(v) for k, v in data.items()}
    elif isinstance(data, list):
        return [clean_and_parse_nested(item) for item in data]
    return data

In [27]:
def get_category_mappings() -> Dict[str, str]:
    """
    Define mappings between attributes and their categories.

    Returns:
        Dictionary mapping attribute keys to their categories
    """
    return {
        "Alcohol": "Special Features/Events",
        "Ambience": "Ambiance",
        "BYOB": "Special Features/Events",
        "BYOBCorkage": "Special Features/Events",
        "BestNights": "Special Features/Events",
        "BikeParking": "Location/Accessibility",
        "BusinessAcceptsBitcoin": "Special Features/Events",
        "BusinessAcceptsCreditCards": "Special Features/Events",
        "BusinessParking": "Location/Accessibility",
        "ByAppointmentOnly": "Special Features/Events",
        "Caters": "Service",
        "CoatCheck": "Special Features/Events",
        "Corkage": "Special Features/Events",
        "DogsAllowed": "Special Features/Events",
        "DriveThru": "Special Features/Events",
        "GoodForDancing": "Special Features/Events",
        "GoodForKids": "Special Features/Events",
        "GoodForMeal": "Food Quality",
        "HappyHour": "Special Features/Events",
        "HasTV": "Special Features/Events",
        "Music": "Ambiance",
        "NoiseLevel": "Ambiance",
        "OutdoorSeating": "Special Features/Events",
        "RestaurantsAttire": "Service",
        "RestaurantsDelivery": "Service",
        "RestaurantsGoodForGroups": "Service",
        "RestaurantsPriceRange2": "Price",
        "RestaurantsReservations": "Reservation Process",
        "RestaurantsTableService": "Service",
        "RestaurantsTakeOut": "Service",
        "Smoking": "Special Features/Events",
        "WheelchairAccessible": "Location/Accessibility",
        "WiFi": "Special Features/Events",
        "BusinessParking - garage": "Location/Accessibility",
        "BusinessParking - lot": "Location/Accessibility",
        "BusinessParking - street": "Location/Accessibility",
        "BusinessParking - valet": "Location/Accessibility",
        "BusinessParking - validated": "Location/Accessibility",
        "Ambience - casual": "Ambiance",
        "Ambience - classy": "Ambiance",
        "Ambience - divey": "Ambiance",
        "Ambience - hipster": "Ambiance",
        "Ambience - intimate": "Ambiance",
        "Ambience - romantic": "Ambiance",
        "Ambience - touristy": "Ambiance",
        "Ambience - trendy": "Ambiance",
        "Ambience - upscale": "Ambiance",
        "GoodForMeal - breakfast": "Food Quality",
        "GoodForMeal - brunch": "Food Quality",
        "GoodForMeal - dessert": "Food Quality",
        "GoodForMeal - dinner": "Food Quality",
        "GoodForMeal - latenight": "Food Quality",
        "GoodForMeal - lunch": "Food Quality",
        "Music - background_music": "Ambiance",
        "Music - dj": "Ambiance",
        "Music - jukebox": "Ambiance",
        "Music - karaoke": "Ambiance",
        "Music - live": "Ambiance",
        "Music - no_music": "Ambiance",
        "Music - video": "Ambiance",
        "BestNights - friday": "Special Features/Events",
        "BestNights - monday": "Special Features/Events",
        "BestNights - saturday": "Special Features/Events",
        "BestNights - sunday": "Special Features/Events",
        "BestNights - thursday": "Special Features/Events",
        "BestNights - tuesday": "Special Features/Events",
        "BestNights - wednesday": "Special Features/Events"
    }

In [28]:
def flatten_attributes_with_categories(
    business_id: str,
    name: str,
    attributes: Dict,
    category_mappings: Dict[str, str]
) -> List[Dict]:
    """
    Flatten nested attributes dictionary into rows with categorization.

    Args:
        business_id: Identifier for the business
        name: Business name
        attributes: Dictionary of business attributes
        category_mappings: Mapping of attributes to categories

    Returns:
        List of dictionaries containing flattened attribute data
    """
    rows = []
    for key, value in attributes.items():
        if isinstance(value, dict):
            for sub_key, sub_value in value.items():
                mapping_key = f"{key} - {sub_key}"
                rows.append({
                    "business_id": business_id,
                    "name": name,
                    "fully_parsed_attribute": attributes,
                    "key": mapping_key,
                    "value": sub_value,
                    "question_category": category_mappings.get(mapping_key, category_mappings.get(key, "Other"))
                })
        else:
            rows.append({
                "business_id": business_id,
                "name": name,
                "fully_parsed_attribute": attributes,
                "key": key,
                "value": value,
                "question_category": category_mappings.get(key, "Other")
            })
    return rows

def process_business_data(df: pd.DataFrame) -> pd.DataFrame:
    """
    Process business data by cleaning attributes and flattening the structure.

    Args:
        df: Input DataFrame with business data

    Returns:
        Processed DataFrame with flattened attribute structure
    """
    df["fully_parsed_attribute"] = df["attributes"].apply(clean_and_parse_nested)
    category_mappings = get_category_mappings()

    final_flattened_data = []
    for _, row in df.iterrows():
        final_flattened_data.extend(flatten_attributes_with_categories(
            row["business_id"],
            row["name"],
            row["fully_parsed_attribute"],
            category_mappings
        ))

    return pd.DataFrame(final_flattened_data)

In [29]:
processed_df = process_business_data(df)

In [30]:
processed_df.head()

,business_id,name,fully_parsed_attribute,key,value,question_category
0,4e9tguRcMc4S5hzBlfC62A,The Farm and Fisherman Tavern,"{'GoodForKids': True, 'RestaurantsGoodForGroups': True, 'RestaurantsReservations': True, 'BusinessAcceptsCreditCards': True, 'RestaurantsAttire': 'casual', 'Alcohol': 'full_bar', 'RestaurantsTakeOut': True, 'RestaurantsPriceRange2': 2, 'HasTV': True, 'BusinessParking': {'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}, 'NoiseLevel': 'average', 'Ambience': {'touristy': False, 'hipster': False, 'romantic': False, 'divey': False, 'intimate': False, 'trendy': False, 'upscale': False, 'classy': True, 'casual': True}, 'HappyHour': True, 'RestaurantsTableService': True, 'WiFi': 'free', 'ByAppointmentOnly': False, 'BikeParking': True, 'WheelchairAccessible': True, 'OutdoorSeating': True, 'Caters': True, 'BYOB': False, 'RestaurantsDelivery': None, 'DogsAllowed': False, 'Corkage': True, 'GoodForMeal': {'dessert': None, 'latenight': False, 'lunch': True, 'dinner': True, 'brunch': None, 'breakfast': False}}",GoodForKids,True,Special Features/Events
1,4e9tguRcMc4S5hzBlfC62A,The Farm and Fisherman Tavern,"{'GoodForKids': True, 'RestaurantsGoodForGroups': True, 'RestaurantsReservations': True, 'BusinessAcceptsCreditCards': True, 'RestaurantsAttire': 'casual', 'Alcohol': 'full_bar', 'RestaurantsTakeOut': True, 'RestaurantsPriceRange2': 2, 'HasTV': True, 'BusinessParking': {'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}, 'NoiseLevel': 'average', 'Ambience': {'touristy': False, 'hipster': False, 'romantic': False, 'divey': False, 'intimate': False, 'trendy': False, 'upscale': False, 'classy': True, 'casual': True}, 'HappyHour': True, 'RestaurantsTableService': True, 'WiFi': 'free', 'ByAppointmentOnly': False, 'BikeParking': True, 'WheelchairAccessible': True, 'OutdoorSeating': True, 'Caters': True, 'BYOB': False, 'RestaurantsDelivery': None, 'DogsAllowed': False, 'Corkage': True, 'GoodForMeal': {'dessert': None, 'latenight': False, 'lunch': True, 'dinner': True, 'brunch': None, 'breakfast': False}}",RestaurantsGoodForGroups,True,Service
2,4e9tguRcMc4S5hzBlfC62A,The Farm and Fisherman Tavern,"{'GoodForKids': True, 'RestaurantsGoodForGroups': True, 'RestaurantsReservations': True, 'BusinessAcceptsCreditCards': True, 'RestaurantsAttire': 'casual', 'Alcohol': 'full_bar', 'RestaurantsTakeOut': True, 'RestaurantsPriceRange2': 2, 'HasTV': True, 'BusinessParking': {'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}, 'NoiseLevel': 'average', 'Ambience': {'touristy': False, 'hipster': False, 'romantic': False, 'divey': False, 'intimate': False, 'trendy': False, 'upscale': False, 'classy': True, 'casual': True}, 'HappyHour': True, 'RestaurantsTableService': True, 'WiFi': 'free', 'ByAppointmentOnly': False, 'BikeParking': True, 'WheelchairAccessible': True, 'OutdoorSeating': True, 'Caters': True, 'BYOB': False, 'RestaurantsDelivery': None, 'DogsAllowed': False, 'Corkage': True, 'GoodForMeal': {'dessert': None, 'latenight': False, 'lunch': True, 'dinner': True, 'brunch': None, 'breakfast': False}}",RestaurantsReservations,True,Reservation Process
3,4e9tguRcMc4S5hzBlfC62A,The Farm and Fisherman Tavern,"{'GoodForKids': True, 'RestaurantsGoodForGroups': True, 'RestaurantsReservations': True, 'BusinessAcceptsCreditCards': True, 'RestaurantsAttire': 'casual', 'Alcohol': 'full_bar', 'RestaurantsTakeOut': True, 'RestaurantsPriceRange2': 2, 'HasTV': True, 'BusinessParking': {'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}, 'NoiseLevel': 'average', 'Ambience': {'touristy': False, 'hipster': False, 'romantic': False, 'divey': False, 'intimate': False, 'trendy': False, 'upscale': False, 'classy': True, 'casual': True}, 'HappyHour': True, 'RestaurantsTableService': True, 'WiFi': 'free', 'ByAppointmentOnly': False, 'BikeParking': True, 'WheelchairAccessible': True, 'OutdoorSeating': True, 'Caters': True, 'BYOB': 

In [31]:
question_mapping = {
    'BusinessAcceptsCreditCards': 'Does {0} accept credit cards?',
    'RestaurantsTakeOut': 'Does {0} offer takeout?',
    'BusinessParking - garage': 'Does {0} have a garage for parking?',
    'BusinessParking - street': 'Does {0} offer on-street parking?',
    'BusinessParking - validated': 'Does {0} offer validated parking?',
    'BusinessParking - lot': 'Does {0} have a parking lot?',
    'BusinessParking - valet': 'Does {0} offer valet parking?',
    'WiFi': 'Does {0} have WiFi available?',
    'RestaurantsReservations': 'Does {0} accept reservations?',
    'RestaurantsAttire': 'What is the attire expected at {0}?',
    'HasTV': 'Does {0} have TVs?',
    'GoodForKids': 'Is {0} good for kids?',
    'NoiseLevel': 'What is the noise level like at {0}?',
    'BikeParking': 'Does {0} have bike parking?',
    'RestaurantsPriceRange2': 'What is the price range at {0}?',
    'Caters': 'Does {0} cater events?',
    'RestaurantsGoodForGroups': 'Is {0} good for large groups?',
    'Alcohol': 'Does {0} serve alcohol?',
    'WheelchairAccessible': 'Is {0} wheelchair accessible?',
    'DogsAllowed': 'Are dogs allowed at {0}?',
    'HappyHour': 'Does {0} offer happy hour specials?',
    'BYOB': 'Is {0} BYOB?',
    'RestaurantsDelivery': 'Does {0} offer delivery?',
    'Corkage': 'Does {0} have a corkage fee?',
    'RestaurantsTableService': 'Does {0} offer table service?',
    'OutdoorSeating': 'Does {0} have outdoor seating?',
    'Ambience - divey': 'Would you describe the ambience at {0} as divey?',
    'Ambience - hipster': 'Would you describe the ambience at {0} as hipster?',
    'Ambience - casual': 'Would you describe the ambience at {0} as casual?',
    'Ambience - touristy': 'Would you describe the ambience at {0} as touristy?',
    'Ambience - trendy': 'Would you describe the ambience at {0} as trendy?',
    'Ambience - intimate': 'Would you describe the ambience at {0} as intimate?',
    'Ambience - romantic': 'Would you describe the ambience at {0} as romantic?',
    'Ambience - classy': 'Would you describe the ambience at {0} as classy?',
    'Ambience - upscale': 'Would you describe the ambience at {0} as upscale?',
    'GoodForMeal - dessert': 'Is {0} good for dessert?',
    'GoodForMeal - latenight': 'Is {0} good for late night dining?',
    'GoodForMeal - lunch': 'Is {0} good for lunch?',
    'GoodForMeal - dinner': 'Is {0} good for dinner?',
    'GoodForMeal - brunch': 'Is {0} good for brunch?',
    'GoodForMeal - breakfast': 'Is {0} good for breakfast?',
    'Smoking': 'Does {0} allow smoking?',
    'CoatCheck': 'Does {0} have a coat check?',
    'Music - dj': 'Does {0} have a DJ?',
    'Music - background_music': 'Does {0} have background music?',
    'Music - no_music': 'Does {0} have no music?',
    'Music - jukebox': 'Does {0} have a jukebox?',
    'Music - live': 'Does {0} have live music?',
    'Music - video': 'Does {0} have music videos?',
    'Music - karaoke': 'Does {0} have karaoke?',
    'BestNights - monday': 'Is Monday a good night to visit {0}?',
    'BestNights - tuesday': 'Is Tuesday a good night to visit {0}?',
    'BestNights - friday': 'Is Friday a good night to visit {0}?',
    'BestNights - wednesday': 'Is Wednesday a good night to visit {0}?',
    'BestNights - thursday': 'Is Thursday a good night to visit {0}?',
    'BestNights - sunday': 'Is Sunday a good night to visit {0}?',
    'BestNights - saturday': 'Is Saturday a good night to visit {0}?',
    'BusinessAcceptsBitcoin': 'Does {0} accept Bitcoin?',
    'GoodForDancing': 'Is {0} good for dancing?',
    'ByAppointmentOnly': 'Is {0} by appointment only?',
    'BYOBCorkage': 'Does {0} have a BYOB corkage fee?',
    'DriveThru': 'Does {0} have a drive-thru?'
}

processed_df['question'] = processed_df.apply(lambda row:
                                     question_mapping.get(row['key'],
                                                         f'No question defined for key: {row["key"]}').format(row['name']),
                                     axis=1)

In [33]:
processed_df.head()

,business_id,name,fully_parsed_attribute,key,value,question_category,question
0,4e9tguRcMc4S5hzBlfC62A,The Farm and Fisherman Tavern,"{'GoodForKids': True, 'RestaurantsGoodForGroups': True, 'RestaurantsReservations': True, 'BusinessAcceptsCreditCards': True, 'RestaurantsAttire': 'casual', 'Alcohol': 'full_bar', 'RestaurantsTakeOut': True, 'RestaurantsPriceRange2': 2, 'HasTV': True, 'BusinessParking': {'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}, 'NoiseLevel': 'average', 'Ambience': {'touristy': False, 'hipster': False, 'romantic': False, 'divey': False, 'intimate': False, 'trendy': False, 'upscale': False, 'classy': True, 'casual': True}, 'HappyHour': True, 'RestaurantsTableService': True, 'WiFi': 'free', 'ByAppointmentOnly': False, 'BikeParking': True, 'WheelchairAccessible': True, 'OutdoorSeating': True, 'Caters': True, 'BYOB': False, 'RestaurantsDelivery': None, 'DogsAllowed': False, 'Corkage': True, 'GoodForMeal': {'dessert': None, 'latenight': False, 'lunch': True, 'dinner': True, 'brunch': None, 'breakfast': False}}",GoodForKids,True,Special Features/Events,Is The Farm and Fisherman Tavern good for kids?
1,4e9tguRcMc4S5hzBlfC62A,The Farm and Fisherman Tavern,"{'GoodForKids': True, 'RestaurantsGoodForGroups': True, 'RestaurantsReservations': True, 'BusinessAcceptsCreditCards': True, 'RestaurantsAttire': 'casual', 'Alcohol': 'full_bar', 'RestaurantsTakeOut': True, 'RestaurantsPriceRange2': 2, 'HasTV': True, 'BusinessParking': {'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}, 'NoiseLevel': 'average', 'Ambience': {'touristy': False, 'hipster': False, 'romantic': False, 'divey': False, 'intimate': False, 'trendy': False, 'upscale': False, 'classy': True, 'casual': True}, 'HappyHour': True, 'RestaurantsTableService': True, 'WiFi': 'free', 'ByAppointmentOnly': False, 'BikeParking': True, 'WheelchairAccessible': True, 'OutdoorSeating': True, 'Caters': True, 'BYOB': False, 'RestaurantsDelivery': None, 'DogsAllowed': False, 'Corkage': True, 'GoodForMeal': {'dessert': None, 'latenight': False, 'lunch': True, 'dinner': True, 'brunch': None, 'breakfast': False}}",RestaurantsGoodForGroups,True,Service,Is The Farm and Fisherman Tavern good for large groups?
2,4e9tguRcMc4S5hzBlfC62A,The Farm and Fisherman Tavern,"{'GoodForKids': True, 'RestaurantsGoodForGroups': True, 'RestaurantsReservations': True, 'BusinessAcceptsCreditCards': True, 'RestaurantsAttire': 'casual', 'Alcohol': 'full_bar', 'RestaurantsTakeOut': True, 'RestaurantsPriceRange2': 2, 'HasTV': True, 'BusinessParking': {'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}, 'NoiseLevel': 'average', 'Ambience': {'touristy': False, 'hipster': False, 'romantic': False, 'divey': False, 'intimate': False, 'trendy': False, 'upscale': False, 'classy': True, 'casual': True}, 'HappyHour': True, 'RestaurantsTableService': True, 'WiFi': 'free', 'ByAppointmentOnly': False, 'BikeParking': True, 'WheelchairAccessible': True, 'OutdoorSeating': True, 'Caters': True, 'BYOB': False, 'RestaurantsDelivery': None, 'DogsAllowed': False, 'Corkage': True, 'GoodForMeal': {'dessert': None, 'latenight': False, 'lunch': True, 'dinner': True, 'brunch': None, 'breakfast': False}}",RestaurantsReservations,True,Reservation Process,Does The Farm and Fisherman Tavern accept reservations?
3,4e9tguRcMc4S5hzBlfC62A,The Farm and Fisherman Tavern,"{'GoodForKids': True, 'RestaurantsGoodForGroups': True, 'RestaurantsReservations': True, 'BusinessAcceptsCreditCards': True, 'RestaurantsAttire': 'casual', 'Alcohol': 'full_bar', 'RestaurantsTakeOut': True, 'RestaurantsPriceRange2': 2, 'HasTV': True, 'BusinessParking': {'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}, 'NoiseLevel': 'average', 'Ambience': {'touristy': False, 'hipster': False, 'romantic': False, 'divey': False, 'intimate': False, 'trendy': False, 'upscale': False, 'classy': True, 'casual': True}, 'HappyHour': True, 'Restau

In [34]:
def get_answer(row):
    answer = ''

    if row['key'] == 'BusinessAcceptsCreditCards':
        answer = 'The restaurant accepts credit cards.' if row['value'] == True else 'The restaurant does not accept credit cards.'
    elif row['key'] == 'RestaurantsTakeOut':
        answer = 'The restaurant offers takeout.' if row['value'] == True else 'The restaurant does not offer takeout.'
    elif row['key'] == 'BusinessParking - garage':
        answer = 'The restaurant has a parking garage.' if row['value'] == True else 'The restaurant does not have a parking garage.'
    elif row['key'] == 'BusinessParking - street':
        answer = 'The restaurant has on-street parking.' if row['value'] == True else 'The restaurant does not have on-street parking.'
    elif row['key'] == 'BusinessParking - validated':
        answer = 'The restaurant offers validated parking.' if row['value'] == True else 'The restaurant does not offer validated parking.'
    elif row['key'] == 'BusinessParking - lot':
        answer = 'The restaurant has a parking lot.' if row['value'] == True else 'The restaurant does not have a parking lot.'
    elif row['key'] == 'BusinessParking - valet':
        answer = 'The restaurant offers valet parking.' if row['value'] == True else 'The restaurant does not offer valet parking.'
    elif row['key'] == 'WiFi':
        if row['value'] == 'free':
            answer = 'The restaurant offers free WiFi.'
        elif row['value'] == 'no':
            answer = 'The restaurant does not offer WiFi.'
        else:
            answer = 'The WiFi availability is unknown.'
    elif row['key'] == 'RestaurantsReservations':
        answer = 'The restaurant accepts reservations.' if row['value'] == True else 'The restaurant does not accept reservations.'
    elif row['key'] == 'RestaurantsAttire':
        if row['value'] == 'dressy':
            answer = 'The expected attire is dressy.'
        elif row['value'] == 'casual':
            answer = 'The expected attire is casual.'
        else:
            answer = 'The expected attire is not specified.'
    elif row['key'] == 'HasTV':
        answer = 'The restaurant has TVs.' if row['value'] == True else 'The restaurant does not have TVs.'
    elif row['key'] == 'GoodForKids':
        answer = 'The restaurant is good for kids.' if row['value'] == True else 'The restaurant is not particularly good for kids.'
    elif row['key'] == 'NoiseLevel':
        if row['value'] == 'average':
            answer = 'The noise level at the restaurant is average.'
        elif row['value'] == 'loud':
            answer = 'The noise level at the restaurant is loud.'
        else:
            answer = 'The noise level at the restaurant is not specified.'
    elif row['key'] == 'BikeParking':
        answer = 'The restaurant has bike parking.' if row['value'] == True else 'The restaurant does not have bike parking.'
    elif row['key'] == 'RestaurantsPriceRange2':
        if row['value'] == 2:
            answer = 'The price range at the restaurant is moderate.'
        elif row['value'] == 3:
            answer = 'The price range at the restaurant is on the higher end.'
        else:
            answer = 'The price range at the restaurant is not specified.'
    elif row['key'] == 'Caters':
        answer = 'The restaurant caters events.' if row['value'] == True else 'The restaurant does not cater events.'
    elif row['key'] == 'RestaurantsGoodForGroups':
        answer = 'The restaurant is good for large groups.' if row['value'] == True else 'The restaurant is not particularly good for large groups.'
    elif row['key'] == 'Alcohol':
        if row['value'] == 'full_bar':
            answer = 'The restaurant has a full bar.'
        elif row['value'] == 'beer_and_wine':
            answer = 'The restaurant serves beer and wine.'
        else:
            answer = 'The restaurant does not serve alcohol.'
    elif row['key'] == 'WheelchairAccessible':
        answer = 'The restaurant is wheelchair accessible.' if row['value'] == True else 'The restaurant is not wheelchair accessible.'
    elif row['key'] == 'DogsAllowed':
        answer = 'Dogs are allowed at the restaurant.' if row['value'] == True else 'Dogs are not allowed at the restaurant.'
    elif row['key'] == 'HappyHour':
        answer = 'The restaurant offers happy hour specials.' if row['value'] == True else 'The restaurant does not offer happy hour specials.'
    elif row['key'] == 'BYOB':
        answer = 'The restaurant allows BYOB.' if row['value'] == True else 'The restaurant does not allow BYOB.'
    elif row['key'] == 'RestaurantsDelivery':
        answer = 'The restaurant offers delivery.' if row['value'] == True else 'The restaurant does not offer delivery.'
    elif row['key'] == 'Corkage':
        answer = 'The restaurant has a corkage fee.' if row['value'] == True else 'The restaurant does not have a corkage fee.'
    elif row['key'] == 'RestaurantsTableService':
        answer = 'The restaurant offers table service.' if row['value'] == True else 'The restaurant does not offer table service.'
    elif row['key'] == 'OutdoorSeating':
        answer = 'The restaurant has outdoor seating.' if row['value'] == True else 'The restaurant does not have outdoor seating.'
    elif row['key'] == 'Ambience - divey':
        answer = 'The ambience at the restaurant is divey.' if row['value'] == True else 'The ambience at the restaurant is not divey.'
    elif row['key'] == 'Ambience - hipster':
        answer = 'The ambience at the restaurant is hipster.' if row['value'] == True else 'The ambience at the restaurant is not hipster.'
    elif row['key'] == 'Ambience - casual':
        answer = 'The ambience at the restaurant is casual.' if row['value'] == True else 'The ambience at the restaurant is not casual.'
    elif row['key'] == 'Ambience - touristy':
        answer = 'The ambience at the restaurant is touristy.' if row['value'] == True else 'The ambience at the restaurant is not touristy.'
    elif row['key'] == 'Ambience - trendy':
        answer = 'The ambience at the restaurant is trendy.' if row['value'] == True else 'The ambience at the restaurant is not trendy.'
    elif row['key'] == 'Ambience - intimate':
        answer = 'The ambience at the restaurant is intimate.' if row['value'] == True else 'The ambience at the restaurant is not intimate.'
    elif row['key'] == 'Ambience - romantic':
        answer = 'The ambience at the restaurant is romantic.' if row['value'] == True else 'The ambience at the restaurant is not romantic.'
    elif row['key'] == 'Ambience - classy':
        answer = 'The ambience at the restaurant is classy.' if row['value'] == True else 'The ambience at the restaurant is not classy.'
    elif row['key'] == 'Ambience - upscale':
        answer = 'The ambience at the restaurant is upscale.' if row['value'] == True else 'The ambience at the restaurant is not upscale.'
    elif row['key'] == 'GoodForMeal - dessert':
        answer = 'The restaurant is good for dessert.' if row['value'] == True else 'The restaurant is not particularly good for dessert.'
    elif row['key'] == 'GoodForMeal - latenight':
        answer = 'The restaurant is good for late night dining.' if row['value'] == True else 'The restaurant is not particularly good for late night dining.'
    elif row['key'] == 'GoodForMeal - lunch':
        answer = 'The restaurant is good for lunch.' if row['value'] == True else 'The restaurant is not particularly good for lunch.'
    elif row['key'] == 'GoodForMeal - dinner':
        answer = 'The restaurant is good for dinner.' if row['value'] == True else 'The restaurant is not particularly good for dinner.'
    elif row['key'] == 'GoodForMeal - brunch':
        answer = 'The restaurant is good for brunch.' if row['value'] == True else 'The restaurant is not particularly good for brunch.'
    elif row['key'] == 'GoodForMeal - breakfast':
        answer = 'The restaurant is good for breakfast.' if row['value'] == True else 'The restaurant is not particularly good for breakfast.'
    elif row['key'] == 'Smoking':
        if row['value'] == 'outdoor':
            answer = 'The restaurant allows outdoor smoking.'
        elif row['value'] == 'no':
            answer = 'The restaurant does not allow smoking.'
        else:
            answer = 'The smoking policy is unknown.'
    elif row['key'] == 'CoatCheck':
        answer = 'The restaurant has a coat check.' if row['value'] == True else 'The restaurant does not have a coat check.'
    elif row['key'] == 'Music - dj':
        answer = 'The restaurant has a DJ.' if row['value'] == True else 'The restaurant does not have a DJ.'
    elif row['key'] == 'Music - background_music':
        answer = 'The restaurant has background music.' if row['value'] == True else 'The restaurant does not have background music.'
    elif row['key'] == 'Music - no_music':
        answer = 'The restaurant has no music.' if row['value'] == True else 'The restaurant has music.'
    elif row['key'] == 'Music - jukebox':
        answer = 'The restaurant has a jukebox.' if row['value'] == True else 'The restaurant does not have a jukebox.'
    elif row['key'] == 'Music - live':
        answer = 'The restaurant has live music.' if row['value'] == True else 'The restaurant does not have live music.'
    elif row['key'] == 'Music - video':
        answer = 'The restaurant has music videos.' if row['value'] == True else 'The restaurant does not have music videos.'
    elif row['key'] == 'Music - karaoke':
        answer = 'The restaurant has karaoke.' if row['value'] == True else 'The restaurant does not have karaoke.'
    elif row['key'] == 'BestNights - monday':
        answer = 'Monday is a good night to visit the restaurant.' if row['value'] == True else 'Monday is not a good night to visit the restaurant.'
    elif row['key'] == 'BestNights - tuesday':
        answer = 'Tuesday is a good night to visit the restaurant.' if row['value'] == True else 'Tuesday is not a good night to visit the restaurant.'
    elif row['key'] == 'BestNights - friday':
        answer = 'Friday is a good night to visit the restaurant.' if row['value'] == True else 'Friday is not a good night to visit the restaurant.'
    elif row['key'] == 'BestNights - wednesday':
        answer = 'Wednesday is a good night to visit the restaurant.' if row['value'] == True else 'Wednesday is not a good night to visit the restaurant.'
    elif row['key'] == 'BestNights - thursday':
        answer = 'Thursday is a good night to visit the restaurant.' if row['value'] == True else 'Thursday is not a good night to visit the restaurant.'
    elif row['key'] == 'BestNights - sunday':
        answer = 'Sunday is a good night to visit the restaurant.' if row['value'] == True else 'Sunday is not a good night to visit the restaurant.'
    elif row['key'] == 'BestNights - saturday':
        answer = 'Saturday is a good night to visit the restaurant.' if row['value'] == True else 'Saturday is not a good night to visit the restaurant.'
    elif row['key'] == 'BusinessAcceptsBitcoin':
        answer = 'The restaurant accepts Bitcoin.' if row['value'] == True else 'The restaurant does not accept Bitcoin.'
    elif row['key'] == 'GoodForDancing':
        answer = 'The restaurant is good for dancing.' if row['value'] == True else 'The restaurant is not good for dancing.'
    elif row['key'] == 'ByAppointmentOnly':
        answer = 'The restaurant is by appointment only.' if row['value'] == True else 'The restaurant is not by appointment only.'
    elif row['key'] == 'BYOBCorkage':
        if row['value'] == 'yes_free':
            answer = 'The restaurant allows BYOB with no corkage fee.'
        else:
            answer = 'The restaurant does not allow BYOB or has a corkage fee.'
    elif row['key'] == 'DriveThru':
        answer = 'The restaurant has a drive-thru.' if row['value'] == True else 'The restaurant does not have a drive-thru.'
    else:
        answer = 'No information available.'

    return answer

processed_df['answer'] = processed_df.apply(get_answer, axis=1)

In [35]:
processed_df = processed_df[['business_id', 'name', 'question_category', 'question', 'answer']]

In [36]:
processed_df['question_category'].value_counts()

,count
question_category,
Ambiance,596
Special Features/Events,583
Location/Accessibility,324
Food Quality,288
Service,283
Reservation Process,49
Price,49


In [38]:
processed_df.shape

(2172, 5)

In [37]:
processed_df.head()

,business_id,name,question_category,question,answer
0,4e9tguRcMc4S5hzBlfC62A,The Farm and Fisherman Tavern,Special Features/Events,Is The Farm and Fisherman Tavern good for kids?,The restaurant is good for kids.
1,4e9tguRcMc4S5hzBlfC62A,The Farm and Fisherman Tavern,Service,Is The Farm and Fisherman Tavern good for large groups?,The restaurant is good for large groups.
2,4e9tguRcMc4S5hzBlfC62A,The Farm and Fisherman Tavern,Reservation Process,Does The Farm and Fisherman Tavern accept reservations?,The restaurant accepts reservations.
3,4e9tguRcMc4S5hzBlfC62A,The Farm and Fisherman Tavern,Special Features/Events,Does The Farm and Fisherman Tavern accept credit cards?,The restaurant accepts credit cards.
4,4e9tguRcMc4S5hzBlfC62A,The Farm and Fisherman Tavern,Service,What is the attire expected at The Farm and Fisherman Tavern?,The expected attire is casual.


In [39]:
hours_df = df[['name', 'business_id', 'hours']]
hours_df = hours_df.drop_duplicates(subset=['business_id'])

In [40]:
hours_df.head()

,name,business_id,hours
0,The Farm and Fisherman Tavern,4e9tguRcMc4S5hzBlfC62A,"{'Monday': '0:0-0:0', 'Tuesday': '11:30-20:0', 'Wednesday': '11:0-15:0', 'Thursday': '11:30-20:0', 'Friday': '11:30-20:0', 'Saturday': '11:30-20:0'}"
1,Dolsot House,-pyobEWdpZ6jUp42f3I2xw,"{'Monday': '0:0-0:0', 'Tuesday': '11:0-21:0', 'Wednesday': '11:0-21:0', 'Thursday': '11:0-21:0', 'Friday': '11:0-21:0', 'Saturday': '11:0-21:0', 'Sunday': '11:0-21:0'}"
2,The Jug Handle Inn,onp9X_9RRAEzA-yqdiSOcA,"{'Monday': '0:0-0:0', 'Tuesday': '11:0-19:0', 'Wednesday': '11:0-0:0', 'Thursday': '20:0-2:0', 'Friday': '11:0-2:0', 'Saturday': '11:0-2:0', 'Sunday': '11:0-0:0'}"
3,Rat's Restaurant,RTwDv2oLE3gzAZlf63OPXA,"{'Wednesday': '11:0-21:0', 'Thursday': '11:0-21:0', 'Friday': '11:0-21:0', 'Saturday': '11:0-21:0', 'Sunday': '11:0-20:0'}"
4,The Pop Shop - Collingswood,UQMUtySn2q9cSObiw5UVsw,"{'Monday': '0:0-0:0', 'Tuesday': '9:0-20:0', 'Wednesday': '9:0-20:0', 'Thursday': '9:0-20:0', 'Friday': '9:0-20:0', 'Saturday': '8:0-20:0', 'Sunday': '8:0-19:0'}"


In [41]:
def generate_service_questions(row):
    name = row['name']
    business_id = row['business_id']
    hours = row['hours']

    questions = [
        f"Is {name} closed on Mondays?",
        f"Is {name} closed on Tuesdays?",
        f"Is {name} closed on Wednesdays?",
        f"Is {name} closed on Thursdays?",
        f"Is {name} closed on Fridays?",
        f"Is {name} closed on Saturdays?",
        f"Is {name} closed on Sundays?",
        f"Is {name} open on Mondays?",
        f"Is {name} open on Tuesdays?",
        f"Is {name} open on Wednesdays?",
        f"Is {name} open on Thursdays?",
        f"Is {name} open on Fridays?",
        f"Is {name} open on Saturdays?",
        f"Is {name} open on Sundays?",
        f"Does {name} offer late-night dinner service?",
        f"Does {name} serve breakfast?",
        f"What are the breakfast hours at {name}?",
        f"Does {name} serve lunch?",
        f"What are the lunch hours at {name}?",
        f"Does {name} serve dinner?",
        f"What are the dinner hours at {name}?",
        f"Does {name} offer brunch?",
        f"What are the brunch hours at {name}?",
        f"When is {name} open on Mondays?",
        f"When is {name} open on Tuesdays?",
        f"When is {name} open on Wednesdays?",
        f"When is {name} open on Thursdays?",
        f"When is {name} open on Fridays?",
        f"When is {name} open on Saturdays?",
        f"When is {name} open on Sundays?",
        f"Are there any differences in the hours of operation at {name} across the week?"
    ]

    return pd.DataFrame({'name': [name] * 31, 'business_id': [business_id] * 31, 'hours': [hours] * 31, 'question': questions})

In [42]:
hours_df = pd.concat([generate_service_questions(row) for _, row in hours_df.iterrows()], ignore_index=True)
hours_df['question_category'] = 'Service'
hours_df.head(5)

,name,business_id,hours,question,question_category
0,The Farm and Fisherman Tavern,4e9tguRcMc4S5hzBlfC62A,"{'Monday': '0:0-0:0', 'Tuesday': '11:30-20:0', 'Wednesday': '11:0-15:0', 'Thursday': '11:30-20:0', 'Friday': '11:30-20:0', 'Saturday': '11:30-20:0'}",Is The Farm and Fisherman Tavern closed on Mondays?,Service
1,The Farm and Fisherman Tavern,4e9tguRcMc4S5hzBlfC62A,"{'Monday': '0:0-0:0', 'Tuesday': '11:30-20:0', 'Wednesday': '11:0-15:0', 'Thursday': '11:30-20:0', 'Friday': '11:30-20:0', 'Saturday': '11:30-20:0'}",Is The Farm and Fisherman Tavern closed on Tuesdays?,Service
2,The Farm and Fisherman Tavern,4e9tguRcMc4S5hzBlfC62A,"{'Monday': '0:0-0:0', 'Tuesday': '11:30-20:0', 'Wednesday': '11:0-15:0', 'Thursday': '11:30-20:0', 'Friday': '11:30-20:0', 'Saturday': '11:30-20:0'}",Is The Farm and Fisherman Tavern closed on Wednesdays?,Service
3,The Farm and Fisherman Tavern,4e9tguRcMc4S5hzBlfC62A,"{'Monday': '0:0-0:0', 'Tuesday': '11:30-20:0', 'Wednesday': '11:0-15:0', 'Thursday': '11:30-20:0', 'Friday': '11:30-20:0', 'Saturday': '11:30-20:0'}",Is The Farm and Fisherman Tavern closed on Thursdays?,Service
4,The Farm and Fisherman Tavern,4e9tguRcMc4S5hzBlfC62A,"{'Monday': '0:0-0:0', 'Tuesday': '11:30-20:0', 'Wednesday': '11:0-15:0', 'Thursday': '11:30-20:0', 'Friday': '11:30-20:0', 'Saturday': '11:30-20:0'}",Is The Farm and Fisherman Tavern closed on Fridays?,Service


In [46]:
def generate_service_answers(row):
    name = row['name']
    hours = eval(row['hours']) if isinstance(row['hours'], str) else row['hours']
    question = row['question']

    def format_time(time_str):
        # Only format time strings that actually represent time ranges
        if '-' not in time_str:
            return time_str
        if time_str == '0:0-0:0':
            return time_str
        start, end = time_str.split('-')
        start_hour, start_min = start.split(':')
        end_hour, end_min = end.split(':')
        return f"{start_hour}:{start_min.zfill(2)}-{end_hour}:{end_min.zfill(2)}"

    def get_day_from_question(question_text):
        if "on" in question_text:
            day_part = question_text.split("on")[1].strip().rstrip('?')
            day = day_part.split()[0].rstrip('s')
            day_mapping = {
                'Mon': 'Monday',
                'Tue': 'Tuesday',
                'Wed': 'Wednesday',
                'Thu': 'Thursday',
                'Fri': 'Friday',
                'Sat': 'Saturday',
                'Sun': 'Sunday'
            }
            return day_mapping.get(day[:3], day)
        return None

    def parse_time(time_str):
        if time_str == '0:0-0:0':
            return (0, 0)
        start, end = time_str.split('-')
        start_hour = int(start.split(':')[0])
        end_hour = int(end.split(':')[0])
        return (start_hour, end_hour)

    def check_time_range(time_str, start_range, end_range):
        if time_str == '0:0-0:0':
            return False
        start, end = parse_time(time_str)
        return start <= end_range and end >= start_range

    # Handle all question types
    if "When is" in question and "open on" in question:
        day = get_day_from_question(question)
        if day:
            if day in hours:
                time = hours[day]
                if time == '0:0-0:0':
                    return f"{name} is closed on {day}"
                return f"{name} is open on {day} from {format_time(time)}"
            return f"{name} is closed on {day}"

    elif "Is" in question and "closed on" in question:
        day = question.split("closed on")[1].strip().rstrip('?').split()[0]
        day = day.rstrip('s')
        is_closed = day not in hours or hours.get(day) == '0:0-0:0'
        return f"{name} is{' ' if is_closed else ' not '}closed on {day}."

    elif "Is" in question and "open on" in question:
        day = question.split("open on")[1].strip().rstrip('?').split()[0]
        day = day.rstrip('s')
        is_open = day in hours and hours.get(day) != '0:0-0:0'
        return f"{name} is{' ' if is_open else ' not '}open on {day}."

    elif "late-night dinner" in question:
        offers_late_night = any(parse_time(time)[1] >= 23 for time in hours.values() if time != '0:0-0:0')
        return f"{name} {'does' if offers_late_night else 'does not'} offer late-night dinner service."

    elif "differences in the hours" in question:
        operating_hours = {day: format_time(time) for day, time in hours.items() if time != '0:0-0:0'}
        if not operating_hours:
            return f"{name} is closed all week."
        elif len(set(operating_hours.values())) == 1:
            return f"{name} maintains consistent hours on days it's open: {next(iter(operating_hours.values()))}."
        return f"{name} has varying hours across the week. Please check specific days for detailed hours."

    # Handle all meal-specific questions
    elif "serve breakfast" in question:
        breakfast_hours = {day: time for day, time in hours.items()
                         if check_time_range(time, 5, 10)}
        return f"{name} {'does' if breakfast_hours else 'does not'} serve breakfast."

    elif "breakfast hours" in question:
        breakfast_hours = {day: format_time(time) for day, time in hours.items()
                         if check_time_range(time, 5, 10)}
        if not breakfast_hours:
            return f"{name} does not serve breakfast."
        return f"Breakfast hours at {name}: " + "; ".join(f"{day}: {time}" for day, time in breakfast_hours.items())

    elif "serve lunch" in question:
        lunch_hours = {day: time for day, time in hours.items()
                      if check_time_range(time, 11, 16)}
        return f"{name} {'does' if lunch_hours else 'does not'} serve lunch."

    elif "lunch hours" in question:
        lunch_hours = {day: format_time(time) for day, time in hours.items()
                      if check_time_range(time, 11, 16)}
        if not lunch_hours:
            return f"{name} does not serve lunch."
        return f"Lunch hours at {name}: " + "; ".join(f"{day}: {time}" for day, time in lunch_hours.items())

    elif "serve dinner" in question:
        dinner_hours = {day: time for day, time in hours.items()
                       if check_time_range(time, 16, 23)}
        return f"{name} {'does' if dinner_hours else 'does not'} serve dinner."

    elif "dinner hours" in question:
        dinner_hours = {day: format_time(time) for day, time in hours.items()
                       if check_time_range(time, 16, 23)}
        if not dinner_hours:
            return f"{name} does not serve dinner."
        return f"Dinner hours at {name}: " + "; ".join(f"{day}: {time}" for day, time in dinner_hours.items())

    elif "offer brunch" in question:
        brunch_hours = {day: time for day, time in hours.items()
                       if day in ['Saturday', 'Sunday']
                       and check_time_range(time, 10, 15)}
        return f"{name} {'does' if brunch_hours else 'does not'} offer brunch."

    elif "brunch hours" in question:
        brunch_hours = {day: format_time(time) for day, time in hours.items()
                       if day in ['Saturday', 'Sunday']
                       and check_time_range(time, 10, 15)}
        if not brunch_hours:
            return f"{name} does not serve brunch."
        return f"Brunch hours at {name}: " + "; ".join(f"{day}: {time}" for day, time in brunch_hours.items())

    return "Unable to generate answer for this question type."

In [47]:
hours_df['answer'] = hours_df.apply(generate_service_answers, axis=1)

In [48]:
hours_df.head()

,name,business_id,hours,question,question_category,answer
0,The Farm and Fisherman Tavern,4e9tguRcMc4S5hzBlfC62A,"{'Monday': '0:0-0:0', 'Tuesday': '11:30-20:0', 'Wednesday': '11:0-15:0', 'Thursday': '11:30-20:0', 'Friday': '11:30-20:0', 'Saturday': '11:30-20:0'}",Is The Farm and Fisherman Tavern closed on Mondays?,Service,The Farm and Fisherman Tavern is closed on Monday.
1,The Farm and Fisherman Tavern,4e9tguRcMc4S5hzBlfC62A,"{'Monday': '0:0-0:0', 'Tuesday': '11:30-20:0', 'Wednesday': '11:0-15:0', 'Thursday': '11:30-20:0', 'Friday': '11:30-20:0', 'Saturday': '11:30-20:0'}",Is The Farm and Fisherman Tavern closed on Tuesdays?,Service,The Farm and Fisherman Tavern is not closed on Tuesday.
2,The Farm and Fisherman Tavern,4e9tguRcMc4S5hzBlfC62A,"{'Monday': '0:0-0:0', 'Tuesday': '11:30-20:0', 'Wednesday': '11:0-15:0', 'Thursday': '11:30-20:0', 'Friday': '11:30-20:0', 'Saturday': '11:30-20:0'}",Is The Farm and Fisherman Tavern closed on Wednesdays?,Service,The Farm and Fisherman Tavern is not closed on Wednesday.
3,The Farm and Fisherman Tavern,4e9tguRcMc4S5hzBlfC62A,"{'Monday': '0:0-0:0', 'Tuesday': '11:30-20:0', 'Wednesday': '11:0-15:0', 'Thursday': '11:30-20:0', 'Friday': '11:30-20:0', 'Saturday': '11:30-20:0'}",Is The Farm and Fisherman Tavern closed on Thursdays?,Service,The Farm and Fisherman Tavern is not closed on Thursday.
4,The Farm and Fisherman Tavern,4e9tguRcMc4S5hzBlfC62A,"{'Monday': '0:0-0:0', 'Tuesday': '11:30-20:0', 'Wednesday': '11:0-15:0', 'Thursday': '11:30-20:0', 'Friday': '11:30-20:0', 'Saturday': '11:30-20:0'}",Is The Farm and Fisherman Tavern closed on Fridays?,Service,The Farm and Fisherman Tavern is not closed on Friday.


In [50]:
hours_df.shape

(1519, 6)

In [51]:
def get_unique_categories(df, category_column='categories'):
    """
    Extract unique categories from a comma-separated category column

    Parameters:
    df: pandas DataFrame containing the category column
    category_column: name of the column containing categories (default: 'categories')

    Returns:
    set: unique categories across all businesses
    """
    all_categories = df[category_column].str.split(',\s*').explode()
    unique_cats = sorted(set(all_categories))
    return unique_cats

In [52]:
unique_categories = get_unique_categories(df)
print("Unique categories:")
for cat in unique_categories:
    print(f"- {cat}")

Unique categories:
- Active Life
- American (New)
- American (Traditional)
- Arts & Entertainment
- Asian Fusion
- Bakeries
- Bars
- Beauty & Spas
- Beer
- Beer Bar
- Botanical Gardens
- Brazilian
- Breakfast & Brunch
- Breweries
- Bubble Tea
- Burgers
- Cafes
- Caribbean
- Caterers
- Cheesesteaks
- Chicken Wings
- Chinese
- Coffee & Tea
- Comfort Food
- Day Spas
- Delis
- Desserts
- Diners
- Egyptian
- Event Planning & Services
- Food
- French
- Gastropubs
- Gluten-Free
- Greek
- Hair Salons
- Halal
- Hotels
- Hotels & Travel
- Indian
- Italian
- Izakaya
- Japanese
- Juice Bars & Smoothies
- Korean
- Laotian
- Latin American
- Lebanese
- Local Flavor
- Massage
- Mediterranean
- Mexican
- Middle Eastern
- Museums
- Music Venues
- Nightlife
- Noodles
- Pakistani
- Parks
- Party & Event Planning
- Pizza
- Pubs
- Ramen
- Religious Organizations
- Restaurants
- Salad
- Sandwiches
- Seafood
- Soup
- Specialty Food
- Sports Bars
- Steakhouses
- Sushi Bars
- Szechuan
- Tacos
- Tea Rooms
- Tep

In [53]:
def create_category_relationships():
    """
    Define comprehensive hierarchical relationships between all categories
    """
    return {
        # ALCOHOLIC BEVERAGES & NIGHTLIFE
        'Bars': {
            'implies': ['Beer', 'Wine & Spirits', 'Nightlife'],
            'subcategories': ['Beer Bar', 'Sports Bars', 'Tiki Bars', 'Wine Bars'],
            'related': ['Pubs', 'Gastropubs']
        },
        'Nightlife': {
            'related': ['Bars', 'Pubs', 'Wine Bars', 'Beer Bar'],
            'implies': ['Restaurants']
        },
        'Breweries': {
            'implies': ['Beer', 'Bars'],
            'related': ['Pubs', 'Beer Bar']
        },
        'Pubs': {
            'implies': ['Beer', 'Bars', 'Food'],
            'related': ['Gastropubs', 'Beer Bar', 'Sports Bars']
        },
        'Gastropubs': {
            'implies': ['Beer', 'Food', 'Restaurants'],
            'related': ['Pubs', 'Bars']
        },

        # CUISINE TYPES
        'American (New)': {
            'implies': ['Restaurants', 'Food'],
            'related': ['American (Traditional)']
        },
        'American (Traditional)': {
            'implies': ['Restaurants', 'Food'],
            'related': ['American (New)']
        },
        'Japanese': {
            'subcategories': ['Izakaya', 'Sushi Bars', 'Teppanyaki', 'Ramen'],
            'implies': ['Noodles', 'Seafood', 'Restaurants']
        },
        'Chinese': {
            'subcategories': ['Szechuan'],
            'implies': ['Noodles', 'Restaurants'],
            'related': ['Soup']
        },
        'Middle Eastern': {
            'implies': ['Halal', 'Restaurants'],
            'related': ['Lebanese', 'Egyptian', 'Mediterranean']
        },
        'Mediterranean': {
            'implies': ['Restaurants'],
            'related': ['Greek', 'Lebanese', 'Middle Eastern']
        },
        'Italian': {
            'implies': ['Restaurants'],
            'related': ['Pizza', 'Pasta']
        },
        'Mexican': {
            'implies': ['Restaurants'],
            'related': ['Tacos', 'Latin American']
        },
        'Latin American': {
            'implies': ['Restaurants'],
            'related': ['Mexican', 'Caribbean']
        },
        'Brazilian': {
            'implies': ['Restaurants'],
            'related': ['Latin American', 'Steakhouses']
        },
        'Caribbean': {
            'implies': ['Restaurants'],
            'related': ['Latin American']
        },
        'Indian': {
            'implies': ['Restaurants'],
            'related': ['Pakistani']
        },
        'Pakistani': {
            'implies': ['Restaurants'],
            'related': ['Indian']
        },
        'Greek': {
            'implies': ['Restaurants'],
            'related': ['Mediterranean']
        },
        'Lebanese': {
            'implies': ['Restaurants', 'Middle Eastern'],
            'related': ['Mediterranean']
        },
        'Egyptian': {
            'implies': ['Restaurants', 'Middle Eastern'],
            'related': ['Mediterranean']
        },
        'Laotian': {
            'implies': ['Restaurants'],
            'related': ['Asian']
        },

        # FOOD TYPES
        'Food': {
            'parent': True,
            'subcategories': ['Bakeries', 'Burgers', 'Cheesesteaks', 'Desserts',
                            'Noodles', 'Pizza', 'Salad', 'Sandwiches', 'Seafood',
                            'Soup', 'Steakhouses', 'Tacos']
        },
        'Steakhouses': {
            'implies': ['Restaurants'],
            'related': ['American (Traditional)', 'Brazilian']
        },

        # DIETARY RESTRICTIONS
        'Vegan': {
            'implies': ['Vegetarian'],
            'related': ['Gluten-Free']
        },
        'Vegetarian': {
            'related': ['Vegan', 'Gluten-Free']
        },
        'Gluten-Free': {
            'related': ['Vegan', 'Vegetarian']
        },
        'Halal': {
            'related': ['Middle Eastern']
        },

        # BEVERAGES
        'Coffee & Tea': {
            'related': ['Cafes', 'Bubble Tea', 'Juice Bars & Smoothies']
        },
        'Bubble Tea': {
            'related': ['Coffee & Tea', 'Asian']
        },
        'Juice Bars & Smoothies': {
            'related': ['Coffee & Tea', 'Healthy']
        },

        # SERVICE TYPES
        'Restaurants': {
            'parent': True,
            'subcategories': ['Cafes', 'Delis', 'Diners'],
            'implies': ['Food']
        },
        'Breakfast & Brunch': {
            'implies': ['Restaurants'],
            'related': ['Cafes', 'Diners']
        },
        'Cafes': {
            'implies': ['Coffee & Tea'],
            'related': ['Breakfast & Brunch']
        },
        'Diners': {
            'implies': ['Restaurants', 'Breakfast & Brunch'],
            'related': ['American (Traditional)']
        },
        'Delis': {
            'implies': ['Sandwiches'],
            'related': ['Cheesesteaks']
        },

        # VENUES & SERVICES
        'Event Planning & Services': {
            'implies': ['Party & Event Planning', 'Venues & Event Spaces'],
            'related': ['Hotels', 'Caterers']
        },
        'Party & Event Planning': {
            'related': ['Event Planning & Services', 'Venues & Event Spaces']
        },
        'Venues & Event Spaces': {
            'related': ['Event Planning & Services', 'Hotels']
        },
        'Hotels': {
            'related': ['Hotels & Travel', 'Venues & Event Spaces']
        },
        'Hotels & Travel': {
            'related': ['Hotels', 'Venues & Event Spaces']
        },
        'Caterers': {
            'related': ['Event Planning & Services', 'Food']
        },
        'Local Flavor': {
            'related': ['Restaurants']
        },

        # WELLNESS & BEAUTY
        'Beauty & Spas': {
            'subcategories': ['Day Spas', 'Hair Salons', 'Massage']
        },
        'Day Spas': {
            'parent': 'Beauty & Spas',
            'related': ['Massage']
        },
        'Hair Salons': {
            'parent': 'Beauty & Spas'
        },
        'Massage': {
            'parent': 'Beauty & Spas',
            'related': ['Day Spas']
        }
    }

In [54]:
def create_question_mapping():
    """
    Question mapping organized by question categories.
    """
    return {
        # FOOD QUALITY
        'American (New)': {'question': 'Does {restaurant} serve New American cuisine?', 'category': 'Popular Dishes'},
        'American (Traditional)': {'question': 'Does {restaurant} serve Traditional American food?', 'category': 'Popular Dishes'},
        'Brazilian': {'question': 'Does {restaurant} serve Brazilian food?', 'category': 'Popular Dishes'},
        'Caribbean': {'question': 'Does {restaurant} serve Caribbean food?', 'category': 'Popular Dishes'},
        'Chinese': {'question': 'Does {restaurant} serve Chinese food?', 'category': 'Popular Dishes'},
        'Egyptian': {'question': 'Does {restaurant} serve Egyptian food?', 'category': 'Popular Dishes'},
        'Greek': {'question': 'Does {restaurant} serve Greek food?', 'category': 'Popular Dishes'},
        'Indian': {'question': 'Does {restaurant} serve Indian food?', 'category': 'Popular Dishes'},
        'Italian': {'question': 'Does {restaurant} serve Italian food?', 'category': 'Popular Dishes'},
        'Izakaya': {'question': 'Does {restaurant} serve Japanese pub food?', 'category': 'Popular Dishes'},
        'Japanese': {'question': 'Does {restaurant} serve Japanese food?', 'category': 'Popular Dishes'},
        'Laotian': {'question': 'Does {restaurant} serve Laotian food?', 'category': 'Popular Dishes'},
        'Latin American': {'question': 'Does {restaurant} serve Latin American food?', 'category': 'Popular Dishes'},
        'Lebanese': {'question': 'Does {restaurant} serve Lebanese food?', 'category': 'Popular Dishes'},
        'Mediterranean': {'question': 'Does {restaurant} serve Mediterranean food?', 'category': 'Popular Dishes'},
        'Mexican': {'question': 'Does {restaurant} serve Mexican food?', 'category': 'Popular Dishes'},
        'Middle Eastern': {'question': 'Does {restaurant} serve Middle Eastern food?', 'category': 'Popular Dishes'},
        'Pakistani': {'question': 'Does {restaurant} serve Pakistani food?', 'category': 'Popular Dishes'},
        'Szechuan': {'question': 'Does {restaurant} serve Szechuan food?', 'category': 'Popular Dishes'},

        # Popular Dishes (Specific Items)
        'Burgers': {'question': 'Does {restaurant} serve burgers?', 'category': 'Popular Dishes'},
        'Cheesesteaks': {'question': 'Does {restaurant} serve cheesesteaks?', 'category': 'Popular Dishes'},
        'Noodles': {'question': 'Does {restaurant} serve noodle dishes?', 'category': 'Popular Dishes'},
        'Pizza': {'question': 'Does {restaurant} serve pizza?', 'category': 'Popular Dishes'},
        'Ramen': {'question': 'Does {restaurant} serve ramen?', 'category': 'Popular Dishes'},
        'Salad': {'question': 'Does {restaurant} serve salads?', 'category': 'Popular Dishes'},
        'Sandwiches': {'question': 'Does {restaurant} serve sandwiches?', 'category': 'Popular Dishes'},
        'Seafood': {'question': 'Does {restaurant} serve seafood?', 'category': 'Popular Dishes'},
        'Soup': {'question': 'Does {restaurant} serve soup?', 'category': 'Popular Dishes'},
        'Steakhouses': {'question': 'Does {restaurant} serve steaks?', 'category': 'Popular Dishes'},
        'Sushi Bars': {'question': 'Does {restaurant} serve sushi?', 'category': 'Popular Dishes'},
        'Tacos': {'question': 'Does {restaurant} serve tacos?', 'category': 'Popular Dishes'},
        'Teppanyaki': {'question': 'Does {restaurant} serve teppanyaki?', 'category': 'Popular Dishes'},
        'Bakeries': {'question': 'Does {restaurant} serve baked goods?', 'category': 'Popular Dishes'},
        'Desserts': {'question': 'Does {restaurant} serve desserts?', 'category': 'Popular Dishes'},


        # Drinks
        'Bars': {'question': 'Does {restaurant} have a bar?', 'category': 'Service'},
        'Beer': {'question': 'Does {restaurant} serve beer?', 'category': 'Service'},
        'Beer Bar': {'question': 'Does {restaurant} have a beer bar?', 'category': 'Service'},
        'Breweries': {'question': 'Is {restaurant} a brewery?', 'category': 'Service'},
        'Bubble Tea': {'question': 'Does {restaurant} serve bubble tea?', 'category': 'Service'},
        'Coffee & Tea': {'question': 'Does {restaurant} serve coffee and tea?', 'category': 'Service'},
        'Juice Bars & Smoothies': {'question': 'Does {restaurant} serve juice and smoothies?', 'category': 'Service'},
        'Wine & Spirits': {'question': 'Does {restaurant} serve wine and spirits?', 'category': 'Service'},
        'Wine Bars': {'question': 'Does {restaurant} have a wine bar?', 'category': 'Service'},
        'Tiki Bars': {'question': 'Does {restaurant} have a tiki bar?', 'category': 'Service'},
        'Sports Bars': {'question': 'Is {restaurant} a sports bar?', 'category': 'Service'},
        'Pubs': {'question': 'Is {restaurant} a pub?', 'category': 'Service'},
        'Gastropubs': {'question': 'Is {restaurant} a gastropub?', 'category': 'Service'},

        # Service Types
        'Breakfast & Brunch': {'question': 'Does {restaurant} serve breakfast or brunch?', 'category': 'Service'},
        'Cafes': {'question': 'Is {restaurant} a cafe?', 'category': 'Service'},
        'Caterers': {'question': 'Does {restaurant} offer catering?', 'category': 'Service'},
        'Delis': {'question': 'Is {restaurant} a deli?', 'category': 'Service'},
        'Diners': {'question': 'Is {restaurant} a diner?', 'category': 'Service'},
        'Restaurants': {'question': 'Is {restaurant} a full-service restaurant?', 'category': 'Service'},

        # Venues & Services
        'Beauty & Spas': {'question': 'Does {restaurant} have spa services?', 'category': 'Special Features/Events'},
        'Day Spas': {'question': 'Does {restaurant} have a day spa?', 'category': 'Special Features/Events'},
        'Event Planning & Services': {'question': 'Does {restaurant} offer event planning?', 'category': 'Special Features/Events'},
        'Hair Salons': {'question': 'Does {restaurant} have a hair salon?', 'category': 'Special Features/Events'},
        'Hotels': {'question': 'Is {restaurant} in a hotel?', 'category': 'Special Features/Events'},
        'Hotels & Travel': {'question': 'Does {restaurant} offer travel services?', 'category': 'Special Features/Events'},
        'Local Flavor': {'question': 'Does {restaurant} feature local specialties?', 'category': 'Special Features/Events'},
        'Massage': {'question': 'Does {restaurant} offer massage services?', 'category': 'Special Features/Events'},
        'Nightlife': {'question': 'Does {restaurant} have nightlife?', 'category': 'Special Features/Events'},
        'Party & Event Planning': {'question': 'Does {restaurant} do event planning?', 'category': 'Special Features/Events'},
        'Venues & Event Spaces': {'question': 'Does {restaurant} have event spaces?', 'category': 'Special Features/Events'},

        # Dietary Options
        'Food': {'question': 'Does {restaurant} serve food?', 'category': 'Special Dietary Requirements'},
        'Gluten-Free': {'question': 'Does {restaurant} have gluten-free options?', 'category': 'Special Dietary Requirements'},
        'Halal': {'question': 'Does {restaurant} serve halal food?', 'category': 'Special Dietary Requirements'},
        'Vegan': {'question': 'Does {restaurant} have vegan options?', 'category': 'Special Dietary Requirements'},
        'Vegetarian': {'question': 'Does {restaurant} have vegetarian options?', 'category': 'Special Dietary Requirements'}
    }

In [55]:
def get_expanded_categories(categories, relationships):
    """
    Expand category list based on relationships and inference rules

    Parameters:
    categories: set of original categories
    relationships: dictionary of category relationships

    Returns:
    set: expanded categories including implied and related categories
    """
    expanded = set(categories)

    while True:
        size_before = len(expanded)

        for category in list(expanded):
            if category in relationships:
                if 'implies' in relationships[category]:
                    expanded.update(relationships[category]['implies'])

                for potential_parent, rel_info in relationships.items():
                    if 'subcategories' in rel_info and category in rel_info['subcategories']:
                        expanded.add(potential_parent)

                if 'subcategories' in relationships[category]:
                    for subcat in relationships[category]['subcategories']:
                        if subcat in expanded:
                            expanded.add(category)

                if 'related' in relationships[category]:
                    for related in relationships[category]['related']:
                        if related in expanded:
                            if related in relationships and 'implies' in relationships[related]:
                                expanded.update(relationships[related]['implies'])

        if len(expanded) == size_before:
            break

    return expanded

In [56]:
import pandas as pd

def generate_restaurant_qa(df):
    """
    Generate questions and answers for restaurants based on their categories

    Parameters:
    df: pandas DataFrame with columns [business_id, name, categories]

    Returns:
    pandas DataFrame with columns [business_id, name, categories, question_category, question, answer]
    """
    questions = create_question_mapping()
    relationships = create_category_relationships()

    results = []

    for _, row in df.iterrows():
        categories_str = str(row['categories']).strip()
        base_categories = {cat.strip() for cat in categories_str.split(',')}

        expanded_categories = get_expanded_categories(base_categories, relationships)

        for category, q_info in questions.items():
            has_category = category in expanded_categories

            question = q_info['question'].format(restaurant=row['name'])

            if q_info['category'] == 'Food Quality':
                answer = generate_food_quality_answer(row['name'], category, has_category)
            elif q_info['category'] == 'Service':
                answer = generate_service_answer(row['name'], category, has_category)
            elif q_info['category'] == 'Ambiance':
                answer = generate_ambiance_answer(row['name'], category, has_category)
            elif q_info['category'] == 'Special Dietary Requirements':
                answer = generate_dietary_answer(row['name'], category, has_category)
            else:
                answer = generate_default_answer(row['name'], category, has_category)

            results.append({
                'business_id': row['business_id'],
                'name': row['name'],
                'categories': row['categories'],
                'question_category': q_info['category'],
                'question': question,
                'answer': answer
            })

    return pd.DataFrame(results)

def generate_food_quality_answer(name, category, has_category):
    if has_category:
        return f"{name} is known for its {category.lower()}. Reviews indicate good quality."
    return f"{name} does not specialize in {category.lower()}."

def generate_service_answer(name, category, has_category):
    if has_category:
        return f"{name} offers {category.lower()} service. Staff is professional."
    return f"{name} does not provide {category.lower()} service."

def generate_ambiance_answer(name, category, has_category):
    if has_category:
        return f"{name} has a pleasant {category.lower()} atmosphere."
    return f"{name} does not feature a {category.lower()} atmosphere."

def generate_dietary_answer(name, category, has_category):
    if has_category:
        return f"{name} accommodates {category.lower()} dietary requirements."
    return f"{name} may not accommodate {category.lower()} dietary requirements."

def generate_default_answer(name, category, has_category):
    if has_category:
        return f"{name} offers {category.lower()}."
    return f"{name} does not offer {category.lower()}."

In [57]:
category_df = generate_restaurant_qa(df)

In [59]:
category_df.head(20)

,business_id,name,categories,question_category,question,answer
0,4e9tguRcMc4S5hzBlfC62A,The Farm and Fisherman Tavern,"Restaurants, American (New)",Popular Dishes,Does The Farm and Fisherman Tavern serve New American cuisine?,The Farm and Fisherman Tavern offers american (new).
1,4e9tguRcMc4S5hzBlfC62A,The Farm and Fisherman Tavern,"Restaurants, American (New)",Popular Dishes,Does The Farm and Fisherman Tavern serve Traditional American food?,The Farm and Fisherman Tavern does not offer american (traditional).
2,4e9tguRcMc4S5hzBlfC62A,The Farm and Fisherman Tavern,"Restaurants, American (New)",Popular Dishes,Does The Farm and Fisherman Tavern serve Brazilian food?,The Farm and Fisherman Tavern does not offer brazilian.
3,4e9tguRcMc4S5hzBlfC62A,The Farm and Fisherman Tavern,"Restaurants, American (New)",Popular Dishes,Does The Farm and Fisherman Tavern serve Caribbean food?,The Farm and Fisherman Tavern does not offer caribbean.
4,4e9tguRcMc4S5hzBlfC62A,The Farm and Fisherman Tavern,"Restaurants, American (New)",Popular Dishes,Does The Farm and Fisherman Tavern serve Chinese food?,The Farm and Fisherman Tavern does not offer chinese.
5,4e9tguRcMc4S5hzBlfC62A,The Farm and Fisherman Tavern,"Restaurants, American (New)",Popular Dishes,Does The Farm and Fisherman Tavern serve Egyptian food?,The Farm and Fisherman Tavern does not offer egyptian.
6,4e9tguRcMc4S5hzBlfC62A,The Farm and Fisherman Tavern,"Restaurants, American (New)",Popular Dishes,Does The Farm and Fisherman Tavern serve Greek food?,The Farm and Fisherman Tavern does not offer greek.
7,4e9tguRcMc4S5hzBlfC62A,The Farm and Fisherman Tavern,"Restaurants, American (New)",Popular Dishes,Does The Farm and Fisherman Tavern serve Indian food?,The Farm and Fisherman Tavern does not offer indian.
8,4e9tguRcMc4S5hzBlfC62A,The Farm and Fisherman Tavern,"Restaurants, American (New)",Popular Dishes,Does The Farm and Fisherman Tavern serve Italian food?,The Farm and Fisherman Tavern does not offer italian.
9,4e9tguRcMc4S5hzBlfC62A,The Farm and Fisherman Tavern,"Restaurants, American (New)",Popular Dishes,Does The Farm and Fisherman Tavern serve Japanese pub food?,The Farm and Fisherman Tavern does not offer izakaya.


In [60]:
category_df.shape

(3381, 6)

In [61]:
def generate_food_quality_questions():
    return [
        "How would you rate the overall food quality at [NAME]?",
        "How consistent is the food preparation and presentation at [NAME] across different visits?",
        "How fresh are the ingredients used in dishes at [NAME]?",
        "How would you rate the portion sizes at [NAME]?",
        "How well does [NAME] accommodate dietary restrictions and modifications?",
        "How would you rate the temperature of hot/cold dishes served at [NAME]?",
        "How visually appealing is the food presentation at [NAME]?",
        "How would you rate the flavor complexity of dishes at [NAME]?",
        "How well-seasoned are the dishes at [NAME]?",
        "How would you rate the menu variety at [NAME]?",
        "How does [NAME] handle special requests for meal modifications?",
        "How would you rate the quality of signature dishes at [NAME]?",
        "How well does [NAME] maintain food quality during busy hours?",
        "How would you rate the quality of side dishes at [NAME]?",
        "How well does [NAME] execute cooking preferences (e.g., meat doneness, spice levels)?",
        "How would you rate the aroma of the dishes served at [NAME]?",
        "How well does [NAME] maintain the quality of their takeout/delivery food?"
    ]

def create_food_quality_questions_df(restaurants_df):
    """
    Create a DataFrame with food quality questions for each restaurant.

    Args:
        restaurants_df: DataFrame containing restaurant information with 'name' column

    Returns:
        DataFrame with questions for each restaurant
    """
    questions = generate_food_quality_questions()

    all_questions = []
    for _, row in restaurants_df.iterrows():
        restaurant_name = row['name']
        for question in questions:
            all_questions.append({
                'business_id': row['business_id'],
                'restaurant_name': restaurant_name,
                'question': question.replace('[NAME]', restaurant_name),
                'category': 'Food Quality'
            })

    return pd.DataFrame(all_questions)

In [62]:
food_quality_questions_df = create_food_quality_questions_df(df)
print(f"Total questions generated: {len(food_quality_questions_df)}")
print(food_quality_questions_df[food_quality_questions_df['business_id'] == df['business_id'].iloc[0]]['question'].head())

Total questions generated: 833
0                                        How would you rate the overall food quality at The Farm and Fisherman Tavern?
1    How consistent is the food preparation and presentation at The Farm and Fisherman Tavern across different visits?
2                                       How fresh are the ingredients used in dishes at The Farm and Fisherman Tavern?
3                                               How would you rate the portion sizes at The Farm and Fisherman Tavern?
4                      How well does The Farm and Fisherman Tavern accommodate dietary restrictions and modifications?
Name: question, dtype: object


In [66]:
def generate_service_questions():
    return [
        "How would you rate the promptness of service at [NAME]?",
        "How knowledgeable are the staff about the menu items at [NAME]?",
        "How attentive are the servers during your dining experience at [NAME]?",
        "How well does [NAME]'s staff handle special requests or modifications?",
        "How would you rate the professionalism of the service staff at [NAME]?",
        "How efficiently does [NAME] handle busy periods and wait times?",
        "How well does the staff at [NAME] coordinate between the kitchen and dining room?",
        "How would you rate the accuracy of orders at [NAME]?",
        "How well does [NAME]'s staff handle customer complaints or issues?",
        "How would you rate the cleanliness and table maintenance at [NAME]?",
        "How effective is the communication between staff and customers at [NAME]?",
        "How well does [NAME] accommodate large groups or special events?",
        "How would you rate the staff's attention to detail at [NAME]?",
        "How consistent is the service quality across different visits to [NAME]?",
        "How well does [NAME]'s staff maintain a welcoming atmosphere?",
        "How efficiently does [NAME] handle takeout and delivery orders?",
        "How would you rate the overall hospitality experience at [NAME]?"
    ]

def create_service_questions_df(restaurants_df):
    """
    Create a DataFrame with service-related questions for each restaurant.

    Args:
        restaurants_df: DataFrame containing restaurant information with 'name' column

    Returns:
        DataFrame with service questions for each restaurant
    """
    questions = generate_service_questions()

    all_questions = []
    for _, row in restaurants_df.iterrows():
        restaurant_name = row['name']
        for question in questions:
            all_questions.append({
                'business_id': row['business_id'],
                'restaurant_name': restaurant_name,
                'question': question.replace('[NAME]', restaurant_name),
                'category': 'Service'
            })

    return pd.DataFrame(all_questions)

In [64]:
service_questions_df = create_service_questions_df(df)

In [65]:
print(f"Total service questions generated: {len(service_questions_df)}")
print("\nSample questions for first restaurant:")
print(service_questions_df[service_questions_df['business_id'] == df['business_id'].iloc[0]]['question'].head())

Total service questions generated: 833

Sample questions for first restaurant:
0                   How would you rate the promptness of service at The Farm and Fisherman Tavern?
1           How knowledgeable are the staff about the menu items at The Farm and Fisherman Tavern?
2    How attentive are the servers during your dining experience at The Farm and Fisherman Tavern?
3    How well does The Farm and Fisherman Tavern's staff handle special requests or modifications?
4    How would you rate the professionalism of the service staff at The Farm and Fisherman Tavern?
Name: question, dtype: object


In [67]:
def generate_ambiance_questions():
    return [
        "How would you rate the overall atmosphere at [NAME]?",
        "How appropriate is the lighting level at [NAME]?",
        "How would you rate the noise level and acoustics at [NAME]?",
        "How comfortable is the seating arrangement at [NAME]?",
        "How well does [NAME]'s decor match its dining concept?",
        "How would you rate the cleanliness of the dining area at [NAME]?",
        "How well does [NAME] maintain comfortable temperature levels?",
        "How would you rate the background music selection at [NAME]?",
        "How spacious is the layout and table spacing at [NAME]?",
        "How well does [NAME]'s ambiance contribute to the dining experience?",
        "How would you rate the ventilation and air quality at [NAME]?",
        "How well does [NAME] maintain its restroom facilities?",
        "How would you rate the visual aesthetics of [NAME]'s interior design?",
        "How consistent is the ambiance throughout different sections of [NAME]?",
        "How well does [NAME] create a suitable atmosphere for its target audience?",
        "How would you rate the outdoor dining atmosphere at [NAME]?",
        "How well does [NAME] balance privacy and social atmosphere?"
    ]

def create_ambiance_questions_df(restaurants_df):
    """
    Create a DataFrame with ambiance-related questions for each restaurant.

    Args:
        restaurants_df: DataFrame containing restaurant information with 'name' column

    Returns:
        DataFrame with ambiance questions for each restaurant
    """
    questions = generate_ambiance_questions()

    all_questions = []
    for _, row in restaurants_df.iterrows():
        restaurant_name = row['name']
        for question in questions:
            all_questions.append({
                'business_id': row['business_id'],
                'restaurant_name': restaurant_name,
                'question': question.replace('[NAME]', restaurant_name),
                'category': 'Ambiance'
            })

    return pd.DataFrame(all_questions)

In [68]:
ambiance_questions_df = create_ambiance_questions_df(df)

print(f"Total ambiance questions generated: {len(ambiance_questions_df)}")
print("\nSample questions for first restaurant:")
print(ambiance_questions_df[ambiance_questions_df['business_id'] == df['business_id'].iloc[0]]['question'].head())

Total ambiance questions generated: 833

Sample questions for first restaurant:
0           How would you rate the overall atmosphere at The Farm and Fisherman Tavern?
1               How appropriate is the lighting level at The Farm and Fisherman Tavern?
2    How would you rate the noise level and acoustics at The Farm and Fisherman Tavern?
3          How comfortable is the seating arrangement at The Farm and Fisherman Tavern?
4         How well does The Farm and Fisherman Tavern's decor match its dining concept?
Name: question, dtype: object


In [74]:
def generate_price_questions():
    return [
        "How would you rate the overall value for money at [NAME]?",
        "How reasonable are the portion sizes relative to prices at [NAME]?",
        "How competitive are [NAME]'s prices compared to similar restaurants in the area?",
        "How well does [NAME] communicate their pricing structure?",
        "How would you rate the quality of food relative to the price at [NAME]?",
        "How transparent is [NAME] about additional charges and fees?",
        "How fair are the prices for specialty items and dishes at [NAME]?",
        "How would you rate the value of combo meals or set menus at [NAME]?",
        "How reasonable are the beverage prices at [NAME]?",
        "How well does [NAME] accommodate different budget preferences?",
        "How would you rate the price consistency across different menu sections at [NAME]?",
        "How fair are the prices for premium ingredients or special requests at [NAME]?",
        "How would you rate the value of promotional offers at [NAME]?",
        "How reasonable are the prices for side dishes and extras at [NAME]?",
        "How well does [NAME] balance quality and price in their menu options?",
        "How would you rate the overall dining experience considering the price at [NAME]?",
        "How fair are the prices for group dining or special events at [NAME]?"
    ]

def create_price_questions_df(restaurants_df):
    """
    Create a DataFrame with price-related questions for each restaurant.

    Args:
        restaurants_df: DataFrame containing restaurant information with 'name' column

    Returns:
        DataFrame with price questions for each restaurant
    """
    questions = generate_price_questions()

    all_questions = []
    for _, row in restaurants_df.iterrows():
        restaurant_name = row['name']
        for question in questions:
            all_questions.append({
                'business_id': row['business_id'],
                'restaurant_name': restaurant_name,
                'question': question.replace('[NAME]', restaurant_name),
                'category': 'Price'
            })

    return pd.DataFrame(all_questions)

In [70]:
price_questions_df = create_price_questions_df(df)

print(f"Total price questions generated: {len(price_questions_df)}")
print("\nSample questions for first restaurant:")
print(price_questions_df[price_questions_df['business_id'] == df['business_id'].iloc[0]]['question'].head())

Total price questions generated: 833

Sample questions for first restaurant:
0                           How would you rate the overall value for money at The Farm and Fisherman Tavern?
1                  How reasonable are the portion sizes relative to prices at The Farm and Fisherman Tavern?
2    How competitive are The Farm and Fisherman Tavern's prices compared to similar restaurants in the area?
3                           How well does The Farm and Fisherman Tavern communicate their pricing structure?
4             How would you rate the quality of food relative to the price at The Farm and Fisherman Tavern?
Name: question, dtype: object


In [73]:
def generate_location_questions():
    return [
        "How easy is it to find [NAME] from main roads or landmarks?",
        "How convenient is the parking situation at [NAME]?",
        "How accessible is [NAME] via public transportation?",
        "How well does [NAME] accommodate wheelchair access?",
        "How visible are the entrance and signage of [NAME]?",
        "How safe is the neighborhood where [NAME] is located?",
        "How easy is it to navigate the interior layout of [NAME]?",
        "How adequate is the parking capacity during peak hours at [NAME]?",
        "How well-lit is the exterior area around [NAME]?",
        "How convenient is [NAME]'s location for different neighborhoods?",
        "How accessible are [NAME]'s restroom facilities?",
        "How well does [NAME] handle curbside pickup or delivery access?",
        "How manageable are the entrance and exit points at [NAME]?",
        "How suitable is the sidewalk access to [NAME]?",
        "How convenient is the drop-off/pick-up area at [NAME]?",
        "How well does [NAME] accommodate stroller or mobility device access?",
        "How easy is it to locate [NAME] using GPS or mapping services?"
    ]

def create_location_questions_df(restaurants_df):
    """
    Create a DataFrame with location and accessibility questions for each restaurant.

    Args:
        restaurants_df: DataFrame containing restaurant information with 'name' column

    Returns:
        DataFrame with location/accessibility questions for each restaurant
    """
    questions = generate_location_questions()

    all_questions = []
    for _, row in restaurants_df.iterrows():
        restaurant_name = row['name']
        for question in questions:
            all_questions.append({
                'business_id': row['business_id'],
                'restaurant_name': restaurant_name,
                'question': question.replace('[NAME]', restaurant_name),
                'category': 'Location/Accessibility'
            })

    return pd.DataFrame(all_questions)

In [72]:
location_questions_df = create_location_questions_df(df)

print(f"Total location questions generated: {len(location_questions_df)}")
print("\nSample questions for first restaurant:")
print(location_questions_df[location_questions_df['business_id'] == df['business_id'].iloc[0]]['question'].head())

Total location questions generated: 833

Sample questions for first restaurant:
0    How easy is it to find The Farm and Fisherman Tavern from main roads or landmarks?
1             How convenient is the parking situation at The Farm and Fisherman Tavern?
2            How accessible is The Farm and Fisherman Tavern via public transportation?
3            How well does The Farm and Fisherman Tavern accommodate wheelchair access?
4            How visible are the entrance and signage of The Farm and Fisherman Tavern?
Name: question, dtype: object


In [75]:
def generate_dietary_questions():
    return [
        "How well does [NAME] accommodate vegetarian dietary requirements?",
        "How clear is the allergen information on [NAME]'s menu?",
        "How well does [NAME] handle gluten-free dietary requests?",
        "How flexible is [NAME] in modifying dishes for dietary restrictions?",
        "How knowledgeable is [NAME]'s staff about ingredients in each dish?",
        "How well does [NAME] accommodate vegan dietary requirements?",
        "How clearly are dairy-free options marked on [NAME]'s menu?",
        "How well does [NAME] prevent cross-contamination for allergy-sensitive diners?",
        "How diverse are the menu options for different dietary restrictions at [NAME]?",
        "How consistently does [NAME] maintain quality in modified dietary dishes?",
        "How well does [NAME] accommodate keto or low-carb dietary needs?",
        "How effectively does [NAME] handle complex dietary combinations?",
        "How well does [NAME] cater to religious dietary requirements?",
        "How reliable is [NAME]'s staff in communicating dietary information?",
        "How well does [NAME] accommodate low-sodium dietary requests?",
        "How clear is the nutritional information provided at [NAME]?",
        "How well does [NAME] handle special dietary requests during busy hours?"
    ]

def create_dietary_questions_df(restaurants_df):
    """
    Create a DataFrame with special dietary requirement questions for each restaurant.

    Args:
        restaurants_df: DataFrame containing restaurant information with 'name' column

    Returns:
        DataFrame with dietary requirement questions for each restaurant
    """
    questions = generate_dietary_questions()

    all_questions = []
    for _, row in restaurants_df.iterrows():
        restaurant_name = row['name']
        for question in questions:
            all_questions.append({
                'business_id': row['business_id'],
                'restaurant_name': restaurant_name,
                'question': question.replace('[NAME]', restaurant_name),
                'category': 'Special Dietary Requirements'
            })

    return pd.DataFrame(all_questions)

In [76]:
dietary_questions_df = create_dietary_questions_df(df)

print(f"Total dietary questions generated: {len(dietary_questions_df)}")
print("\nSample questions for first restaurant:")
print(dietary_questions_df[dietary_questions_df['business_id'] == df['business_id'].iloc[0]]['question'].head())

Total dietary questions generated: 833

Sample questions for first restaurant:
0       How well does The Farm and Fisherman Tavern accommodate vegetarian dietary requirements?
1                 How clear is the allergen information on The Farm and Fisherman Tavern's menu?
2               How well does The Farm and Fisherman Tavern handle gluten-free dietary requests?
3    How flexible is The Farm and Fisherman Tavern in modifying dishes for dietary restrictions?
4     How knowledgeable is The Farm and Fisherman Tavern's staff about ingredients in each dish?
Name: question, dtype: object


In [77]:
def generate_popular_dishes_questions():
    return [
        "How would you rate [NAME]'s most popular dish?",
        "How consistent is the quality of signature dishes at [NAME]?",
        "How well does [NAME] present their specialty items?",
        "How would you rate the variety of popular dishes at [NAME]?",
        "How unique are the signature dishes at [NAME] compared to other restaurants?",
        "How well does [NAME] maintain the availability of their popular items?",
        "How would you rate the value for money of signature dishes at [NAME]?",
        "How well does [NAME] execute their chef's special dishes?",
        "How creative are the popular menu items at [NAME]?",
        "How well does [NAME] describe their specialty dishes on the menu?",
        "How seasonal are the popular dishes at [NAME]?",
        "How well does [NAME] incorporate local flavors in their popular dishes?",
        "How would you rate the portion sizes of signature dishes at [NAME]?",
        "How well does [NAME] pair recommendations with their popular dishes?",
        "How authentic are the traditional popular dishes at [NAME]?",
        "How well does [NAME] maintain the quality of popular dishes during peak hours?",
        "How innovative are the signature dishes at [NAME]?"
    ]

def create_popular_dishes_questions_df(restaurants_df):
    """
    Create a DataFrame with popular dishes questions for each restaurant.

    Args:
        restaurants_df: DataFrame containing restaurant information with 'name' column

    Returns:
        DataFrame with popular dishes questions for each restaurant
    """
    questions = generate_popular_dishes_questions()

    all_questions = []
    for _, row in restaurants_df.iterrows():
        restaurant_name = row['name']
        for question in questions:
            all_questions.append({
                'business_id': row['business_id'],
                'restaurant_name': restaurant_name,
                'question': question.replace('[NAME]', restaurant_name),
                'category': 'Popular Dishes'
            })

    return pd.DataFrame(all_questions)

In [78]:
popular_dishes_questions_df = create_popular_dishes_questions_df(df)

print(f"Total popular dishes questions generated: {len(popular_dishes_questions_df)}")
print("\nSample questions for first restaurant:")
print(popular_dishes_questions_df[popular_dishes_questions_df['business_id'] == df['business_id'].iloc[0]]['question'].head())

Total popular dishes questions generated: 833

Sample questions for first restaurant:
0                                  How would you rate The Farm and Fisherman Tavern's most popular dish?
1                    How consistent is the quality of signature dishes at The Farm and Fisherman Tavern?
2                             How well does The Farm and Fisherman Tavern present their specialty items?
3                     How would you rate the variety of popular dishes at The Farm and Fisherman Tavern?
4    How unique are the signature dishes at The Farm and Fisherman Tavern compared to other restaurants?
Name: question, dtype: object


In [79]:
def generate_wait_times_questions():
    return [
        "How reasonable are the wait times during peak hours at [NAME]?",
        "How accurate are the estimated wait times provided by [NAME]?",
        "How efficient is [NAME]'s reservation system?",
        "How well does [NAME] handle walk-in customers during busy periods?",
        "How would you rate the speed of seating at [NAME]?",
        "How effectively does [NAME] manage their waiting list?",
        "How well does [NAME] communicate delays or longer wait times?",
        "How efficient is the check-in process at [NAME]?",
        "How well does [NAME] accommodate large groups in terms of waiting time?",
        "How reasonable are the wait times for takeout orders at [NAME]?",
        "How efficiently does [NAME] clear and prepare tables for waiting customers?",
        "How well does [NAME] handle wait times during special events or holidays?",
        "How would you rate the waiting area comfort at [NAME]?",
        "How effectively does [NAME] use technology for queue management?",
        "How well does [NAME] handle priority seating requests?",
        "How consistent are the service times once seated at [NAME]?",
        "How well does [NAME] balance quick service with dining experience?"
    ]

def create_wait_times_questions_df(restaurants_df):
    """
    Create a DataFrame with wait times questions for each restaurant.

    Args:
        restaurants_df: DataFrame containing restaurant information with 'name' column

    Returns:
        DataFrame with wait times questions for each restaurant
    """
    questions = generate_wait_times_questions()

    all_questions = []
    for _, row in restaurants_df.iterrows():
        restaurant_name = row['name']
        for question in questions:
            all_questions.append({
                'business_id': row['business_id'],
                'restaurant_name': restaurant_name,
                'question': question.replace('[NAME]', restaurant_name),
                'category': 'Wait Times'
            })

    return pd.DataFrame(all_questions)

In [80]:
wait_times_questions_df = create_wait_times_questions_df(df)

print(f"Total wait times questions generated: {len(wait_times_questions_df)}")
print("\nSample questions for first restaurant:")
print(wait_times_questions_df[wait_times_questions_df['business_id'] == df['business_id'].iloc[0]]['question'].head())

Total wait times questions generated: 833

Sample questions for first restaurant:
0        How reasonable are the wait times during peak hours at The Farm and Fisherman Tavern?
1         How accurate are the estimated wait times provided by The Farm and Fisherman Tavern?
2                         How efficient is The Farm and Fisherman Tavern's reservation system?
3    How well does The Farm and Fisherman Tavern handle walk-in customers during busy periods?
4                    How would you rate the speed of seating at The Farm and Fisherman Tavern?
Name: question, dtype: object


In [81]:
def generate_reservation_questions():
    return [
        "How easy is it to make a reservation at [NAME]?",
        "How reliable is [NAME]'s online reservation system?",
        "How flexible is [NAME] with modification of existing reservations?",
        "How well does [NAME] handle special requests during reservation?",
        "How efficient is [NAME]'s confirmation process for reservations?",
        "How reasonable is [NAME]'s cancellation policy?",
        "How well does [NAME] accommodate last-minute reservations?",
        "How effectively does [NAME] handle group reservations?",
        "How clear is [NAME]'s policy about reservation time limits?",
        "How well does [NAME] maintain reserved tables during peak hours?",
        "How efficient is the check-in process for reserved tables at [NAME]?",
        "How well does [NAME] communicate their reservation requirements?",
        "How fair is [NAME]'s deposit policy for reservations?",
        "How well does [NAME] handle holiday or special event reservations?",
        "How effectively does [NAME] manage their waitlist when fully reserved?",
        "How well does [NAME] accommodate preferred seating requests?",
        "How consistent is [NAME] in honoring reserved times?"
    ]

def create_reservation_questions_df(restaurants_df):
    """
    Create a DataFrame with reservation process questions for each restaurant.

    Args:
        restaurants_df: DataFrame containing restaurant information with 'name' column

    Returns:
        DataFrame with reservation process questions for each restaurant
    """
    questions = generate_reservation_questions()

    all_questions = []
    for _, row in restaurants_df.iterrows():
        restaurant_name = row['name']
        for question in questions:
            all_questions.append({
                'business_id': row['business_id'],
                'restaurant_name': restaurant_name,
                'question': question.replace('[NAME]', restaurant_name),
                'category': 'Reservation Process'
            })

    return pd.DataFrame(all_questions)

In [82]:
reservation_questions_df = create_reservation_questions_df(df)

print(f"Total reservation questions generated: {len(reservation_questions_df)}")
print("\nSample questions for first restaurant:")
print(reservation_questions_df[reservation_questions_df['business_id'] == df['business_id'].iloc[0]]['question'].head())


Total reservation questions generated: 833

Sample questions for first restaurant:
0                       How easy is it to make a reservation at The Farm and Fisherman Tavern?
1                   How reliable is The Farm and Fisherman Tavern's online reservation system?
2    How flexible is The Farm and Fisherman Tavern with modification of existing reservations?
3      How well does The Farm and Fisherman Tavern handle special requests during reservation?
4      How efficient is The Farm and Fisherman Tavern's confirmation process for reservations?
Name: question, dtype: object


In [83]:
def generate_special_features_questions():
    return [
        "How well does [NAME] organize special holiday events?",
        "How would you rate the quality of live entertainment at [NAME]?",
        "How effectively does [NAME] handle private events or parties?",
        "How unique are [NAME]'s seasonal menu offerings?",
        "How well does [NAME] execute themed dining experiences?",
        "How would you rate [NAME]'s happy hour program?",
        "How engaging are the special promotional events at [NAME]?",
        "How well does [NAME] accommodate celebration events?",
        "How would you rate the quality of catering services at [NAME]?",
        "How creative are [NAME]'s special dining experiences?",
        "How well does [NAME] manage outdoor dining features?",
        "How effectively does [NAME] handle wine tasting events?",
        "How would you rate [NAME]'s loyalty program benefits?",
        "How well does [NAME] incorporate seasonal decorations?",
        "How engaging are [NAME]'s cooking demonstrations or classes?",
        "How well does [NAME] execute special breakfast/brunch events?",
        "How effectively does [NAME] promote their special events?"
    ]

def create_special_features_questions_df(restaurants_df):
    """
    Create a DataFrame with special features and events questions for each restaurant.

    Args:
        restaurants_df: DataFrame containing restaurant information with 'name' column

    Returns:
        DataFrame with special features questions for each restaurant
    """
    questions = generate_special_features_questions()

    all_questions = []
    for _, row in restaurants_df.iterrows():
        restaurant_name = row['name']
        for question in questions:
            all_questions.append({
                'business_id': row['business_id'],
                'restaurant_name': restaurant_name,
                'question': question.replace('[NAME]', restaurant_name),
                'category': 'Special Features/Events'
            })

    return pd.DataFrame(all_questions)

In [84]:
special_features_questions_df = create_special_features_questions_df(df)

print(f"Total special features questions generated: {len(special_features_questions_df)}")
print("\nSample questions for first restaurant:")
print(special_features_questions_df[special_features_questions_df['business_id'] == df['business_id'].iloc[0]]['question'].head())

Total special features questions generated: 833

Sample questions for first restaurant:
0              How well does The Farm and Fisherman Tavern organize special holiday events?
1    How would you rate the quality of live entertainment at The Farm and Fisherman Tavern?
2      How effectively does The Farm and Fisherman Tavern handle private events or parties?
3                   How unique are The Farm and Fisherman Tavern's seasonal menu offerings?
4            How well does The Farm and Fisherman Tavern execute themed dining experiences?
Name: question, dtype: object


In [85]:
all_questions_df = pd.concat([
    food_quality_questions_df,
    service_questions_df,
    ambiance_questions_df,
    price_questions_df,
    location_questions_df,
    dietary_questions_df,
    popular_dishes_questions_df,
    wait_times_questions_df,
    reservation_questions_df,
    special_features_questions_df
])

In [86]:
all_questions_df.head()

,business_id,restaurant_name,question,category
0,4e9tguRcMc4S5hzBlfC62A,The Farm and Fisherman Tavern,How would you rate the overall food quality at The Farm and Fisherman Tavern?,Food Quality
1,4e9tguRcMc4S5hzBlfC62A,The Farm and Fisherman Tavern,How consistent is the food preparation and presentation at The Farm and Fisherman Tavern across different visits?,Food Quality
2,4e9tguRcMc4S5hzBlfC62A,The Farm and Fisherman Tavern,How fresh are the ingredients used in dishes at The Farm and Fisherman Tavern?,Food Quality
3,4e9tguRcMc4S5hzBlfC62A,The Farm and Fisherman Tavern,How would you rate the portion sizes at The Farm and Fisherman Tavern?,Food Quality
4,4e9tguRcMc4S5hzBlfC62A,The Farm and Fisherman Tavern,How well does The Farm and Fisherman Tavern accommodate dietary restrictions and modifications?,Food Quality


In [87]:
all_questions_df.shape

(8330, 4)

In [88]:
qs_dataset_final = pd.concat([hours_df, processed_df, category_df, all_questions_df])

In [90]:
qs_dataset_final.head()

,name,business_id,hours,question,question_category,answer,categories,restaurant_name,category
0,The Farm and Fisherman Tavern,4e9tguRcMc4S5hzBlfC62A,"{'Monday': '0:0-0:0', 'Tuesday': '11:30-20:0', 'Wednesday': '11:0-15:0', 'Thursday': '11:30-20:0', 'Friday': '11:30-20:0', 'Saturday': '11:30-20:0'}",Is The Farm and Fisherman Tavern closed on Mondays?,Service,The Farm and Fisherman Tavern is closed on Monday.,NaN,NaN,NaN
1,The Farm and Fisherman Tavern,4e9tguRcMc4S5hzBlfC62A,"{'Monday': '0:0-0:0', 'Tuesday': '11:30-20:0', 'Wednesday': '11:0-15:0', 'Thursday': '11:30-20:0', 'Friday': '11:30-20:0', 'Saturday': '11:30-20:0'}",Is The Farm and Fisherman Tavern closed on Tuesdays?,Service,The Farm and Fisherman Tavern is not closed on Tuesday.,NaN,NaN,NaN
2,The Farm and Fisherman Tavern,4e9tguRcMc4S5hzBlfC62A,"{'Monday': '0:0-0:0', 'Tuesday': '11:30-20:0', 'Wednesday': '11:0-15:0', 'Thursday': '11:30-20:0', 'Friday': '11:30-20:0', 'Saturday': '11:30-20:0'}",Is The Farm and Fisherman Tavern closed on Wednesdays?,Service,The Farm and Fisherman Tavern is not closed on Wednesday.,NaN,NaN,NaN
3,The Farm and Fisherman Tavern,4e9tguRcMc4S5hzBlfC62A,"{'Monday': '0:0-0:0', 'Tuesday': '11:30-20:0', 'Wednesday': '11:0-15:0', 'Thursday': '11:30-20:0', 'Friday': '11:30-20:0', 'Saturday': '11:30-20:0'}",Is The Farm and Fisherman Tavern closed on Thursdays?,Service,The Farm and Fisherman Tavern is not closed on Thursday.,NaN,NaN,NaN
4,The Farm and Fisherman Tavern,4e9tguRcMc4S5hzBlfC62A,"{'Monday': '0:0-0:0', 'Tuesday': '11:30-20:0', 'Wednesday': '11:0-15:0', 'Thursday': '11:30-20:0', 'Friday': '11:30-20:0', 'Saturday': '11:30-20:0'}",Is The Farm and Fisherman Tavern closed on Fridays?,Service,The Farm and Fisherman Tavern is not closed on Friday.,NaN,NaN,NaN


In [91]:
qs_dataset_final['name'] = qs_dataset_final['name'].fillna(qs_dataset_final['restaurant_name'])
qs_dataset_final = qs_dataset_final.drop(columns=['hours', 'categories', 'restaurant_name', 'category'])

In [92]:
qs_dataset_final.head()

,name,business_id,question,question_category,answer
0,The Farm and Fisherman Tavern,4e9tguRcMc4S5hzBlfC62A,Is The Farm and Fisherman Tavern closed on Mondays?,Service,The Farm and Fisherman Tavern is closed on Monday.
1,The Farm and Fisherman Tavern,4e9tguRcMc4S5hzBlfC62A,Is The Farm and Fisherman Tavern closed on Tuesdays?,Service,The Farm and Fisherman Tavern is not closed on Tuesday.
2,The Farm and Fisherman Tavern,4e9tguRcMc4S5hzBlfC62A,Is The Farm and Fisherman Tavern closed on Wednesdays?,Service,The Farm and Fisherman Tavern is not closed on Wednesday.
3,The Farm and Fisherman Tavern,4e9tguRcMc4S5hzBlfC62A,Is The Farm and Fisherman Tavern closed on Thursdays?,Service,The Farm and Fisherman Tavern is not closed on Thursday.
4,The Farm and Fisherman Tavern,4e9tguRcMc4S5hzBlfC62A,Is The Farm and Fisherman Tavern closed on Fridays?,Service,The Farm and Fisherman Tavern is not closed on Friday.


In [93]:
qs_dataset_final.to_csv('all_questions.csv', index=False)